In [1]:
import pandas as pd
pd.options.display.max_columns = 500 


In [2]:
from operator import index
from wsgiref import validate
import pandas as pd 
from config import path
from dics import gcons, f12_vars, f3_vars, f11_vars, siebel_vars, ro_vars, mc_vars, en_vars, q_vars, siebel_vars, tesor_nc, tesor_sieb, pos_sac, pos_cyc, pos_what, columns_res_sac_nnt_nss, q_man_vars, tiendas
from os import listdir
from unidecode import unidecode
from datetime import datetime
import numpy as np 
pd.options.display.max_columns = 500

class CONC:

    dfs = []
    dt = datetime.now().strftime('%y%m%d')

    def  __init__(self) -> None:
        dcorte = input('Ingrese corte: ')
        self.path_input = path + '/input' + '/'+ dcorte 
        self.path_output = path + '/output'
        files = listdir(self.path_input)
        self.file_name_list = ['f3', 'f11', 'f12', 'q_man', 'ro', 'mc', 'en', 'quiebres', 'ss', 'tesoreria_ntc', 'tesoreria_sieb']
        self.file_name_dirs = get_dirs(files, self.file_name_list)

    def run(self):
        # Run 
        print('Cargando archivos ...')
        self.load_files()
        print('Transformando archivos ...')
        self.transform_files()
        print('Realizando uniones ...')
        self.res = self.get_join()
        print('Clasificando archivos ...')
        df = self.f12_classifier()
        print('Guardando archivos ...')
        self.set_up_exit(df)
        res = input('¿Distribuir bases? (y/n)')
        if res == 'y':
            self.save_results(df)
        print('Finalizado')

    def load_files(self):
        self.dfs = []
        for dir in self.file_name_dirs: 
            if dir != '':
                self.dfs.append(pd.read_csv(f'{self.path_input}/{dir}', sep=';', dtype='object'))
            else:
                self.dfs.append(pd.DataFrame())

    def transform_files(self):
        col_key = [f3_vars['key'], f11_vars['key'], f12_vars['key'], q_man_vars['key'], ro_vars['key'], mc_vars['key'], en_vars['key'], q_vars['key'], siebel_vars['key'], tesor_nc['key'], tesor_sieb['key']]
        cols_dup = [f3_vars['dkeys'], f11_vars['dkeys'], f12_vars['dkeys'], q_man_vars['dkeys'], ro_vars['dkeys'], mc_vars['dkeys'], en_vars['dkeys'], q_vars['dkeys'], siebel_vars['dkeys'], tesor_nc['dkeys'], tesor_sieb['dkeys']]
        cols_num = [f3_vars['cnum'], f11_vars['cnum'], f12_vars['cnum'], q_man_vars['cnum'], ro_vars['cnum'], mc_vars['cnum'], en_vars['cnum'], q_vars['cnum'], siebel_vars['cnum'], tesor_nc['cnum'], tesor_sieb['cnum']]
        cols_string = [f3_vars['cstring'], f11_vars['cstring'], f12_vars['cstring'], q_man_vars['cstring'], ro_vars['cstring'], mc_vars['cstring'], en_vars['cstring'], q_vars['cstring'], siebel_vars['cstring'], tesor_nc['cstring'], tesor_sieb['cstring']]
        cols_to_keep = [f3_vars['ckeep'], f11_vars['ckeep'], f12_vars['ckeep'], q_man_vars['ckeep'], ro_vars['ckeep'], mc_vars['ckeep'], en_vars['ckeep'], q_vars['ckeep'], siebel_vars['ckeep'], tesor_nc['ckeep'], tesor_sieb['ckeep']]

        for i, df in enumerate(self.dfs):
            if df.empty == False: 
                self.dfs[i] = set_prefijo_df(self.dfs[i], self.file_name_list[i])
                self.dfs[i] = keep_cols(self.dfs[i], cols_to_keep[i])
                self.dfs[i] = clean_num(self.dfs[i], cols_num[i])
                self.dfs[i] = clean_string(self.dfs[i], cols_string[i])
                self.dfs[i] = delete_null_values(self.dfs[i], col_key[i])
                self.dfs[i] = delete_duplicates(self.dfs[i], cols_dup[i])             
        
        # Transformaciones adicionales 
        self.dfs[2].loc[:, f12_vars['fpactada']] = pd.to_datetime(self.dfs[2][f12_vars['fpactada']], format='%Y-%m-%d')
        cond_fpactada = (self.dfs[2][f12_vars['fpactada']]< datetime.now() )& (self.dfs[2][f12_vars['fpactada']].notna())
        self.dfs[2].loc[cond_fpactada, f12_vars['ind_fpactada']] = 'OVERDUE'
        self.dfs[2].loc[~cond_fpactada, f12_vars['ind_fpactada']] = 'ON TIME'
        self.dfs[2].loc[self.dfs[2][f12_vars['nc']] == '0', f12_vars['nc']] = np.nan

    def get_dfs(self):
        return self.dfs 

    def f12_classifier(self):

        f12_pce = pd.read_csv('input/f12_pce.csv', dtype=str)
        
        df = self.res.copy()

        df.reset_index(inplace=True)

        set_pos(df, pos_cyc, 'POS de C&C')
        set_pos(df, pos_what, 'POS de whatsapp')
        set_pos(df, pos_sac, 'POS de SAC')
        df.loc[(df['f12_loc_nc'].isin(tiendas)) & (df['gco_ind_pos'].isna()), 'gco_ind_pos' ] = 'POS piso de venta'

        df.loc[df['f12_nfolio'].isin(f12_pce['NRO_F12']), 'ind_pce' ] = 'y'

        df.loc[df['f12_gco_ind_nc_inst'].notna(), 'ind_solucion_tienda'] = 'y'
        df.loc[df['f12_gco_ind_gift_card'].notna(),'ind_solucion_tienda'] = 'y' 
        df.loc[(df['f12_gco_ind_te'].notna()) & (df['gco_ind_pos'].isin(['POS piso de venta', 'POS de whatsapp'])), 'ind_solucion_tienda'] = 'y'
        
        val_entregas = (df[f3_vars['key']].notna()) | (df[f11_vars['key']].notna()) | (df[mc_vars['key']].notna()) | (df[en_vars['key']].notna()) | (df[q_vars['key']].notna())|(df[ro_vars['key']].notna())
        df.loc[val_entregas, 'gco_ind_entregas'] = 'Tiene registro RO | MC | F3 | F11 | QUIEBRE'

        df.loc[df[siebel_vars['key']].notna() , 'gco_ind_ss'] = 'Tiene SS'
        df.loc[df[siebel_vars['n3']].str.contains(r'TROCADO|AVERIA|INCOMPLETO|ENTREGA FALSO|ENTREGA FALSA', na = False), 'gco_ind_ss_n3'] = 'Se encontró TROCADO|AVERIA|INCOMPLETO|ENTREGA FALSA en SS N3'

        # Se encontro en base tesoreria
        df.loc[(df[tesor_nc['cod_aut']].notna()) | ((df[tesor_sieb['sieb_ss']].notna())), 'gco_ind_btes'] = 'La NC|SS existe en DB tesorería' 
        df.loc[df['gco_ind_btes'].isna(), 'gco_ind_btes' ] = 'La NC|SS no existe en DB tesorería'

        # Unificacion estados, base tesoreria
        df.loc[df[tesor_nc['estado']].isna(), tesor_nc['estado']] = df.loc[df[tesor_nc['estado']].isna(), tesor_sieb['estado_sieb']].values
        df.rename(columns = {tesor_nc['estado']: 'estado_tesoreria'}, inplace=True)

        # Se realizo el pago
        df.loc[((df[tesor_nc['cod_aut']].notna()) | ((df[tesor_sieb['sieb_ss']].notna()))) & (df['estado_tesoreria'] == 'PROCESADA'), 'gco_ind_registra_pago'] = 'Pago procesado'
        df.loc[((df[tesor_nc['cod_aut']].notna()) | ((df[tesor_sieb['sieb_ss']].notna()))) & (df['gco_ind_registra_pago'].isna()), 'gco_ind_registra_pago' ] = 'Pago rechazado'


        total_entrega = get_id_values(df, 'index', f12_vars['estado'], ['TOTAL ENTREGA'])

        # C1 ENTREGA ADMINISTRATIVA
        te_ent_admin = get_id_values(df, 'index', f12_vars['subestado'], ['ENTREGA ADMINISTRATIVO'], total_entrega)
        mts_val = ['EN PROCESO NC', 'ENTREGADO EN TIENDA', 'FALTA STOCK', 'RETORNADO A ORIGEN', 'REFACTURACION', 'PROD CON NCRD']
        c1 = get_id_values(df, 'index', f12_vars['mt'], mts_val, te_ent_admin)

        # C2 ENTREGAS         
        c2 = get_id_values(df, 'index', 'gco_ind_entregas', ['Tiene registro RO | MC | F3 | F11 | QUIEBRE'], total_entrega)


        # C3 NO ENTREGAS 
        no_entregas = [ 'ENTREGA EN TIENDA','EN LINEA PRV. CON FACTURA', 'EN LINEA PRV. PEND. FACTURA', 'TOTAL ENTREGA']
        te_no_entregas = get_id_values(df,'index', f12_vars['subestado'], no_entregas, total_entrega)
        c3 =  get_id_values(df,'index', 'gco_ind_ss_n3', ['Se encontró TROCADO|AVERIA|INCOMPLETO|ENTREGA FALSA en SS N3'], te_no_entregas)

        # C4 ANULADO POR NCRD
        ancdr_est = get_id_values(df,'index', f12_vars['estado'], ['ANULADO X NCRD'] )
        c4 = get_id_values(df,'index', f12_vars['subestado'], ['ANULADO X NCRD'], ancdr_est)

        # C5 EN RUTA Y DIGITADO 
        ## EN RUTA
        en_ruta = get_id_values(df,'index', f12_vars['estado'], ['EN RUTA'] )
        er_se_values = ['EN RUTA AL CLIENTE', 'MT PROVEEDOR', 'EN TRANSITO', 'MT']
        er_se = get_id_values(df,'index', f12_vars['subestado'], er_se_values , en_ruta)
        c5_p1 = get_id_values(df,'index', f12_vars['ind_fpactada'], ['OVERDUE'], er_se)

        ## DIGITADO
        digitados = get_id_values(df,'index', f12_vars['estado'], ['DIGITADO'] )
        dig_se_values = ['BOLETEADO/FACTURADO', 'MT PROVEEDOR']
        dig_se = get_id_values(df,'index', f12_vars['subestado'], dig_se_values, digitados)
        c5_p2 = get_id_values(df,'index', f12_vars['ind_fpactada'], ['OVERDUE'], dig_se)

        # RESULTADO 
        f12s_validos = c1 + c2 + c3 + c4 + c5_p1 + c5_p2
        
        return set_colvalue(df, 'index', f12s_validos, 'gco_comment', 'Aplica conciliación')
    
    def get_join(self):

        # Uniones previas 
        self.dfs[1][f11_vars['f11_f12']] = self.dfs[1][f11_vars['observ_f11']].str.extract(r'(?!S{2} [1]-[1][2])(^[1][2]\d{7,})') 
        self.dfs[1].loc[self.dfs[1][f11_vars['f11_f12']].isna(), f11_vars['f11_f12']] = self.dfs[1].loc[self.dfs[1][f11_vars['f11_f12']].isna(), f11_vars['observ_f11']].str.extract(r'(?!S{2} [1]-[1][2])( [1][2]\d{7,})').values
        self.dfs[1][f11_vars['f11_f12']] = self.dfs[1][f11_vars['f11_f12']].str.strip()
        self.dfs[1] = self.dfs[1].loc[~self.dfs[1][f11_vars['f11_f12']].isna()].reset_index(drop=True)
        self.dfs[1].drop_duplicates([f11_vars['f11_f12'], f11_vars['upc_f11']], inplace=True)
        ne_ro = join( self.dfs[4], self.dfs[6],  ro_vars['ro'], en_vars['centrada'], 'many_to_one') 
        print('union 1 ')

        # Uniendo base de SS por sub orden y nro f12
        f12_ss_1 = join(self.dfs[2], self.dfs[8].loc[(self.dfs[8]['ss_suborden'].notna()) & (self.dfs[8]['ss_suborden'] != 0)], f12_vars['so'], siebel_vars['sorden'], 'many_to_many')
        ss_x_f12 = self.dfs[8].drop_duplicates([siebel_vars['f12']]) 
        f12_ss = f12_ss_1.merge(self.dfs[8].loc[(self.dfs[8]['ss_num_f12'].notna()) & (self.dfs[8]['ss_num_f12'] != 0)], how = 'left', left_on = f12_vars['folio'],right_on = siebel_vars['f12'], suffixes=('', '_y')) 
        f12_ss.loc[(f12_ss[siebel_vars['ss_y']].notna()), siebel_vars['ckeep'] ] = f12_ss.loc[(f12_ss[siebel_vars['ss_y']].notna()), gcons['union_ss_aux']].values
        f12_ss.drop(columns = gcons['union_ss_aux'], inplace = True) # Se eliminan columnas de segunda Base
        f12_ss.drop_duplicates(['f12_nfolio','f12_prd_upc'], inplace=True) #TODO revisar cual ss es la verdadera del f12
        print('union 2 ')

        # Uniendo base tesoreria_nc por cautoriza y ss 
        # f12_tes_ntc_1 = join(f12_ss, self.dfs[9], f12_vars['nc'], tesor_nc['cod_aut'], 'many_to_one')
        # tes_ntc_x_ss = self.dfs[9].drop_duplicates([tesor_nc['ntc_ss']]) 
        # f12_tes_ntc = f12_tes_ntc_1.merge(tes_ntc_x_ss, how = 'left', left_on = siebel_vars['f12'], right_on = tesor_nc['ntc_ss'], suffixes=('', '_y')) 
        # f12_tes_ntc.loc[(f12_tes_ntc[tesor_nc['ntc_ss_y']].notna()), tesor_nc['ckeep']] = f12_tes_ntc.loc[(f12_tes_ntc[tesor_nc['ntc_ss_y']].notna()), gcons['union_tes_ntc_aux']].values
        # f12_tes_ntc.drop(columns = gcons['union_tes_ntc_aux'], inplace = True) # Se eliminan columnas de segunda Base
        # #Continuan unioines

        f12_tes_ntc_1 = join(f12_ss, self.dfs[9], f12_vars['nc'], tesor_nc['cod_aut'], 'many_to_one')
        f12_tes_ss = join(f12_tes_ntc_1, self.dfs[10], siebel_vars['ss'], tesor_sieb['sieb_ss'], 'many_to_one')
        print('union 3 ')
        
        f12_ro = join(f12_tes_ss, ne_ro, [f12_vars['folio'], f12_vars['upc']] , [ro_vars['do_inicial'], ro_vars['upc_ro']], 'one_to_one')
        f12_mc = join(f12_ro, self.dfs[5], f12_vars['folio'] , mc_vars['entrada_mc'], 'many_to_one')
        f12_f11 = join(f12_mc, self.dfs[1], [f12_vars['folio'], f12_vars['upc']] , [f11_vars['f11_f12'], f11_vars['upc_f11']], 'one_to_one')
        f12_f3 = join(f12_f11, self.dfs[0], [f12_vars['folio'], f12_vars['upc']], [f3_vars['f3_f12'], f3_vars['upc_f3']], 'one_to_one')
        f12_q = join(f12_f3, self.dfs[7], [f12_vars['folio'], f12_vars['upc']], [q_vars['f12_q'], q_vars['q_ean']], 'one_to_one')
        self.dfs[3] = self.dfs[3].drop_duplicates(['q_man_f12', 'q_man_sku']) # TODO Validar fuente de datos
        f12_q_m = join(f12_q, self.dfs[3], [f12_vars['folio'], f12_vars['upc']], [q_man_vars['f12'], q_man_vars['sku']], 'one_to_one')
        return f12_q_m

    def set_up_exit(self, df):
        res_ant = pd.read_csv('C:/Users/maperezr/OneDrive - Falabella/General/DATA/CONC/pbi/resultado_agg.csv', sep=';', dtype='object')
        df = df.loc[(df['f12_nfolio'].isin(res_ant['f12_nfolio'].unique())) & (df['f12_dcreacion']<'2022-07-31')]
        locales = pd.read_csv('C:/Users/maperezr/OneDrive - Falabella/General/DATA/listado_locales.csv', dtype = 'object', usecols = ['LOC_ID','LOCAL_AGG'])
        df = df.merge(locales, how='left', left_on = 'f12_loc_id', right_on = 'LOC_ID')
        df.loc[df['gco_ind_ss'].isna(), 'gco_ind_ss'] = 'No tiene SS'
        df.loc[df['gco_comment'].isna(), 'gco_comment'] = 'No aplíca conciliación'
        gco_cols = ['gco_ind_entregas', 'gco_ind_ss', 'gco_ind_ss_n3', 'gco_comment', 'gco_ind_btes', 'estado_tesoreria', 'gco_ind_registra_pago', 'LOCAL_AGG']
        return df 
        # df[f12_vars['ckeep']+gco_cols].to_csv(f'{self.path_output}/{self.dt}_resultado.csv', sep=';', index=False).
    
    def save_results(self, df):
        df['f12_nfolio'] = pd.to_numeric(df['f12_nfolio'])
        res_sac = pd.read_excel('C:/Users/maperezr/Downloads/pablo.xlsx', dtype='object', usecols = ['f12_nfolio', 'comentarios_resp_rev'])
        res_sac.rename(columns={'comentarios_resp_rev':'comentarios_resp_rev_ant'}, inplace=True)
        res_sac['f12_nfolio'] = pd.to_numeric(res_sac['f12_nfolio'])
        nac_snc_sss =  df.loc[(df['f12_gco_inc_nc'] == 'Tiene NC') & (df['gco_comment'] == 'No aplíca conciliación') & (df['f12_desc_estado'] == 'TOTAL ENTREGA') & (df['gco_ind_ss'] == 'Tiene SS')].reset_index(drop = True)
        nac_snc_sss = nac_snc_sss.merge(res_sac, how='left', on='f12_nfolio')
        print('No aplica conciliacion, Tiene NC, Tiene SS')
        print(f'F12\'s {nac_snc_sss.f12_nfolio.nunique()}')
        print(f'SKU {nac_snc_sss.shape[0]}')
        nac_snc_sss.to_excel(f'output/{self.dt}_e1_nac_snc_sss.xlsx', index= False)

        sac_nnc = df.loc[(df['f12_gco_inc_nc'] == 'No tiene NC') & (df['gco_comment'] == 'Aplica conciliación')].reset_index(drop =True)
        print('Aplica conciliacion, No tiene NC')
        print(f'F12\'s {sac_nnc.f12_nfolio.nunique()}')
        print(f'SKU {sac_nnc.shape[0]}')

        sac_nnc_sss = sac_nnc.loc[(sac_nnc['gco_ind_ss'] == 'Tiene SS') & (sac_nnc['ss_fcr'] == 'N')].reset_index(drop =True)
        print('Aplica conciliacion, No tiene NC, Tiene SS')
        print(f'F12\'s {sac_nnc_sss.f12_nfolio.nunique()}')
        print(f'SKU {sac_nnc_sss.shape[0]}')
        sac_nnc_sss.to_excel(f'output/{self.dt}_e2_sac_nnc_sss.xlsx', index= False)

        print('Aplica conciliacion,  No tiene NC, No tiene SS')
        sac_nnc_nss = sac_nnc.loc[(sac_nnc['gco_ind_ss'] == 'No tiene SS')].reset_index(drop =True)
        usr = pd.read_excel('input/220509_usr_locales (2).xlsx', usecols=['Usuario','Local agg'])
        usr.rename(columns={'Local agg':'Local usr cierre'}, inplace = True)
        sac_nnc = sac_nnc.reindex(columns_res_sac_nnt_nss)
        sac_nnc_nss = sac_nnc_nss.merge(usr, how = 'left', left_on='f12_username', right_on = 'Usuario')
        print(f'F12\'s {sac_nnc_nss.f12_nfolio.nunique()}')
        print(f'SKU {sac_nnc_nss.shape[0]}')

        sac_nnc_nss_cd =  sac_nnc_nss.loc[sac_nnc_nss['Local usr cierre'] == 'CD'].reset_index(drop =True)
        print('Aplica conciliacion, No tiene NC, No tiene SS, CD')
        print(f'F12\'s {sac_nnc_nss_cd.f12_nfolio.nunique()}')
        print(f'SKU {sac_nnc_nss_cd.shape[0]}')
        sac_nnc_nss_cd.to_excel(f'output/{self.dt}_e3_sac_nnc_nss_cd.xlsx', index= False)

        sac_nnc_nss_tiendas =  sac_nnc_nss.loc[sac_nnc_nss['Local usr cierre'] == 'TIENDA'].reset_index(drop =True)
        print('Aplica conciliacion, No tiene NC, No tiene SS, Tienda')
        print(f'F12\'s {sac_nnc_nss_tiendas.f12_nfolio.nunique()}')
        print(f'SKU {sac_nnc_nss_tiendas.shape[0]}')
        sac_nnc_nss_tiendas.to_excel(f'output/{self.dt}_e4_sac_nnc_nss_tiendas.xlsx', index= False)

        sac_nnc_nss_sistemas =  sac_nnc_nss.loc[sac_nnc_nss['Local usr cierre'] == 'SISTEMAS'].reset_index(drop =True)
        print('Aplica conciliacion, No tiene NC, No tiene SS, Sistemas')
        print(f'F12\'s {sac_nnc_nss_sistemas.f12_nfolio.nunique()}')
        print(f'SKU {sac_nnc_nss_sistemas.shape[0]}')
        sac_nnc_nss_sistemas.to_excel(f'output/{self.dt}_e5_sac_nnc_nss_sistemas.xlsx', index= False)


def get_id_values(df, id_col, col, values, init_ids = []):
    todos = df[id_col].unique() if len(init_ids) == 0 else init_ids
    return list(df.loc[df[id_col].isin(todos) &  df[col].isin(values), id_col].unique())

def set_colvalue(df, id_col, ids,  comment_col, comment_value): 
    res = df.copy()
    res.loc[res[id_col].isin(ids), comment_col] = comment_value
    return res

def set_prefijo_df(df, prefijo): # TODO reescribir en list coprehension
    col = {}
    for i in df.columns:
        col[i] = f'{prefijo}_{i.lower()}'
    return df.rename(columns=col)

def delete_null_values(df, col): # PAP [x]
    return df.loc[~df[col].isna()].reset_index(drop=True)

def delete_duplicates(df, col_list): # MAP
    return df.drop_duplicates(col_list)

def clean_num(df, col_list): # PAP
    res = df.copy()
    for col in col_list:
        res.loc[:, col] = res[col].str.replace(r'([^0-9,])', '', regex=True)
        res.loc[:, col] = pd.to_numeric(res.loc[:, col] )
    return res

def clean_string(df, col_list): # MAP
    res = df.copy()
    for col in col_list:
        res.loc[res[col].notna(), col] = res.loc[res[col].notna(), col].apply(unidecode)
        res.loc[res[col].notna(), col] = res.loc[res[col].notna(), col].str.replace(r'([^a-zA-Z0-9-+(). ])', '', regex=True)
        res.loc[res[col].notna(), col] = res.loc[res[col].notna(), col].str.strip()
        res.loc[res[col].notna(), col] = res.loc[res[col].notna(), col].str.upper()
    return res

def keep_cols(df, cols_to_keep):
    return df.loc[:, cols_to_keep].reset_index(drop=True)

def get_dirs(files, file_name_list): # PAP [x]
    lista = []
    for name in file_name_list:
        flag = False 
        for file in files:
            if file.__contains__(name): 
                lista.append(file)
                flag = True
        if flag: 
            continue 
        else: 
            lista.append('')
    return lista 

def set_pos(df,dic_pos, result):
    for i in dic_pos:
        df.loc[(df['f12_gco_inc_nc'] == 'Tiene NC') & (df['f12_loc_nc'] == i) & (df['f12_nterminal'].isin(dic_pos[i])), 'gco_ind_pos' ] = result  

def join(df_left, df_right, col_df_left, col_df_right, val_type):
    return df_left.merge(df_right, how = 'left', left_on = col_df_left, right_on = col_df_right, validate=val_type)

def innit_commandline():
    conc = CONC()
    conc.run()

In [3]:
221124

221124

In [4]:
conc = CONC()

In [5]:
print('Cargando archivos ...')
conc.load_files()
print('Transformando archivos ...')
conc.transform_files()
print('Realizando uniones ...')
conc.res = conc.get_join()
print('Clasificando archivos ...')
df = conc.f12_classifier()
print('Guardando archivos ...')
df_res = conc.set_up_exit(df)
# res = input('¿Distribuir bases? (y/n)')
# if res == 'y':
#     conc.save_results(df_res)
# print('Finalizado')

Cargando archivos ...
Transformando archivos ...
Realizando uniones ...
union 1 
union 2 
union 3 
Clasificando archivos ...
Guardando archivos ...


In [6]:
df_res_1 = conc.set_up_exit(df)


In [24]:
df_res = df_res_1.copy()

In [25]:
# TODO Poner esta logica en el clasificador
df_res.loc[(df_res['tesoreria_ntc_valor de devolucion'].notna()) & (df_res['tesoreria_sieb_valor de devolucion'].notna()) & (df_res['tesoreria_sieb_estado'] == 'PROCESADA') & (df_res['estado_tesoreria'] == 'PROCESADA'), 'comentaroio_tes'] = '¿Posible doble devoliució?'

In [26]:
# TODO Generar metodo para analisis de tesoreia
gb_cautoriza = df_res.loc[(df_res['f12_gco_inc_nc'] == 'Tiene C') & (df_res['gco_comment'] == 'Aplica conciliación') & (df_res['tesoreria_sieb_estado'] == 'PROCESADA') |(df_res['estado_tesoreria'] == 'PROCESADA')].groupby(['f12_nfolio', 'f12_cautoriza']).agg({'f12_tprecio':'sum', 'f12_monto_nc':'unique', 'f12_mdescuento':'sum', 'tesoreria_ntc_valor de devolucion':'unique','tesoreria_sieb_valor de devolucion':'sum'}).reset_index()
gb_cautoriza['f12_monto_nc'] = gb_cautoriza['f12_monto_nc'].apply(lambda x : abs(pd.to_numeric(x[0])))
gb_cautoriza['tesoreria_ntc_valor de devolucion'] = gb_cautoriza['tesoreria_ntc_valor de devolucion'].apply(lambda x : abs(pd.to_numeric(x[0])))
gb_cautoriza['f12_total'] = gb_cautoriza['f12_monto_nc'] - gb_cautoriza['f12_mdescuento']
gb_cautoriza['dif_ca'] = abs(gb_cautoriza['f12_total'] - gb_cautoriza['tesoreria_ntc_valor de devolucion'])
gb_cautoriza['dif_ss'] = abs(gb_cautoriza['f12_total'] - gb_cautoriza['tesoreria_sieb_valor de devolucion'])
gb_cautoriza.loc[(gb_cautoriza['dif_ca'] <= 1000) | (gb_cautoriza['dif_ss'] <= 1000), 'dif'] = 0
gb_cautoriza['f12_mdescuento'] = abs(gb_cautoriza['f12_mdescuento'])
gb_cautoriza.loc[(gb_cautoriza['f12_mdescuento'] == gb_cautoriza['dif']) & (gb_cautoriza['f12_mdescuento'] !=0), 'dif' ] = 0.0 
f12_coin = gb_cautoriza.loc[gb_cautoriza['dif'] <= 1000 ,'f12_cautoriza'].unique()
f12_nocoin = gb_cautoriza.loc[gb_cautoriza['dif'] > 1000 ,'f12_cautoriza'].unique()
df_res.loc[(df_res['f12_cautoriza'].isin(f12_coin)) & (df_res['gco_ind_registra_pago'] == 'Pago procesado'), 'comentaroio_tes'] = 'Coindicencia exacta'
df_res.loc[(df_res['f12_cautoriza'].isin(f12_nocoin)) & (df_res['gco_ind_registra_pago'] == 'Pago procesado'),'comentaroio_tes'] = 'No Coindicide valor'
df_res.loc[(df_res['f12_gco_ind_te'].notna()) & (df_res['f12_gco_inc_nc'] == 'Tiene NC') & (df_res['gco_ind_pos'].isin(['POS piso de venta','POS de whatsapp'])), 'Novedad tesoreria'] =  'Solución en tienda'
df_res.loc[(df_res['f12_gco_inc_nc'] == 'Tiene NC') & ((df_res['f12_gco_ind_canal'] != 'Otro canal de venta') | (df_res['f12_gco_ind_nc_inst'].notna()) | (df_res['f12_gco_ind_gift_card'].notna()) | (df_res['ind_pce'].notna()) | ((df_res['f12_cmatrimonio'].notna()) & (df_res['f12_ccomuna'] == '370'))), 'Novedad tesoreria' ] =  'Solución en tienda'
df_res.loc[(df_res['f12_gco_inc_nc'] == 'Tiene NC') & (df_res['gco_comment'] == 'Aplica conciliación') & (df_res['Novedad tesoreria'] !=  'Solución en tienda') & (df_res['gco_ind_btes'] == 'La NC|SS no existe en DB tesorería') , 'Novedad tesoreria' ] = 'No se encontro en base de tesoreria'
df_res.loc[(df_res['f12_gco_inc_nc'] == 'Tiene NC') & (df_res['gco_comment'] == 'No aplíca conciliación') & (df_res['gco_ind_btes'] == 'La NC|SS existe en DB tesorería') & (df_res['gco_ind_registra_pago'] == 'Pago procesado'), 'Novedad tesoreria'] = "No aplica NC pero registra pago tesoreria"
df_res.loc[(df_res['f12_gco_inc_nc'] == 'Tiene NC') & (df_res['gco_comment'] == 'Aplica conciliación') & (df_res['gco_ind_btes'] == 'La NC|SS existe en DB tesorería')  & (df_res['f12_gco_ind_canal'] != 'Otro canal de venta') & (df_res['gco_ind_registra_pago'] == 'Pago procesado'), 'Novedad tesoreria'] = "Canal de venta linio"
df_res.loc[df_res['comentaroio_tes'] == 'No Coindicide valor','Novedad tesoreria'] = "No Coindicide valor"

In [32]:
df_res.groupby('Novedad tesoreria')['f12_nfolio'].nunique()

Novedad tesoreria
Canal de venta linio                            14
No aplica NC pero registra pago tesoreria     5560
No se encontro en base de tesoreria           3681
Solución en tienda                           35918
Name: f12_nfolio, dtype: int64

_____________________

In [28]:
gco_cols = ['gco_ind_entregas', 'gco_ind_ss', 'gco_ind_ss_n3', 'gco_comment' ,'gco_ind_btes','LOCAL_AGG' ,'comentaroio_tes','Novedad tesoreria','gco_ind_registra_pago']

In [29]:
df_res[f12_vars['ckeep'] + gco_cols].to_csv(f'{conc.path_output}/{conc.dt}_resultado.csv', sep=';', index=False)

In [30]:
len(df_res[f12_vars['ckeep'] + gco_cols].columns)

51

__________

_________________

In [36]:
mp = pd.read_csv('input\mp.csv')

C:\Users\maperezr\AppData\Local\Temp\ipykernel_17824\1107772381.py:1: DtypeWarning: Columns (12,27,34,46,48,49) have mixed types. Specify dtype option on import or set low_memory=False.
  mp = pd.read_csv('input\mp.csv')


In [64]:
df_res.loc[(df_res['f12_desc_subestado'] == 'ANULADO X NCRD') & (df_res['f12_desc_subestado'] == 'ANULADO X NCRD') & (df_res['f12_gco_inc_nc'] == 'No tiene NC')]#  & (df_res['f12_ccomuna'] != '370')]['f12_nfolio'].nunique()

,index,f12_nfolio,f12_loc_id,f12_loc_name,f12_dcreacion,f12_caja_venta,f12_secuencia_venta,f12_dreparto,f12_dpactada,f12_ctipo,f12_cvendedor,f12_bretira_dsp,f12_origin,f12_fpestado,f12_pestado,f12_fuestado,f12_desc_estado,f12_desc_subestado,f12_desc_mt,f12_username,f12_prd_upc,f12_prod_sku_id,f12_qproducto,f12_mprecio,f12_tprecio,f12_mdescuento,f12_dproceso,f12_loc_nc,f12_nterminal,f12_nsecuencia,f12_monto_nc,f12_cautoriza,f12_trn_tech_key,f12_origin_nc,f12_nsuborden,f12_ccomuna,f12_cmatrimonio,f12_ctipificacion,f12_gco_inc_nc,f12_gco_ind_canal,f12_gco_ind_nc_inst,f12_gco_ind_gift_card,f12_gco_ind_te,gco_ind_fpactada,ss_ss,ss_n1,ss_n2,ss_n3,ss_tipo,ss_fcr,ss_estado,ss_subestado,ss_fecha_creacion,ss_fecha_solucion,ss_fecha_cierre,ss_tienda_origen,ss_orden_compra,ss_suborden,ss_num_f12,ss_area_problema,ss_sucursal_creador,ss_descripcion,ss_solucion,tesoreria_ntc_cod aut nc,tesoreria_ntc_valor de devolucion,estado_tesoreria,tesoreria_sieb_ss,tesoreria_sieb_valor de devolucion,tesoreria_sieb_estado,ro_do_inicial,ro_upc,ro_fecha_creacion_ro,ro_ro,ro_estado_ro,ro_cantidad_producto,en_centrada,en_fcreareg,en_creferen,en_darticul,en_qcantida,mc_tip0_trabajo,mc_entrada,f11_nro_f11,f11_fecha_creacion,f11_upc,f11_propietario,f11_cod_local,f11_nombre_local,f11_grupo,f11_estado,f11_servicio,f11_cant_prod,f11_total_costo,f11_observacion,f11_folio_f12,f3_nro_devolucion,f3_fecha_reserva,f3_fecha_envio,f3_fecha_anulacion,f3_fecha_confirmacion,f3_tipo_producto,f3_upc,f3_sku,f3_descripcion,f3_local,f3_descripcion5,f3_estado,f3_descripcion6,f3_cantidad,f3_cant*costoprmd,f3_folio_f12,quiebres_f12,quiebres_sku,quiebres_cantidad_cancelada,quiebres_cost_unit_prod,quiebres_tipo_quiebre,quiebres_codigo_barras,q_man_fecha del quiebre,q_man_f12,q_man_sku,q_man_unidades,gco_ind_pos,ind_pce,ind_solucion_tienda,gco_ind_entregas,gco_ind_ss,gco_ind_ss_n3,gco_ind_btes,gco_ind_registra_pago,gco_comment,LOC_ID,LOCAL_AGG,comentaroio_tes,Novedad tesoreria
155,155,12695940912,2000,TIENDA FALABELLA .COM,2022-06-28 17:02:59,1,8564,2022-06-30 00:00:00,2022-06-30,1,47708,F,HDREF,2022-06-28 17:03:05,RESERVADO,2022-07-02 00:32:51,ANULADO X NCRD,ANULADO X NCRD,NaN,EXPSRXCO,50875822979,3607131,1.0,329900.0,329900.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,144029166015,370,288871,NaN,No tiene NC,Otro canal de venta,NaN,NaN,NaN,OVERDUE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No tiene SS,NaN,La NC|SS no existe en DB tesorería,NaN,Aplica conciliación,2000,CD,NaN,NaN
821,825,12697403415,2000,TIENDA FALABELLA .COM,2022-07-14 13:09:06,1,9139,2022-07-16 00:00:00,2022-07-16,1,47708,F,HDREF,2022-07-14 13:09:12,RESERVADO,2022-07-16 00:24:36,ANULADO X NCRD,ANULADO X NCRD,NaN,EXPSRXCO,72244788204,1941066,1.0,198900.0,198900.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,144029374698,370,282218,NaN,No tiene NC,Otro canal de venta,NaN,NaN,NaN,OVERDUE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No tiene SS,NaN,La NC|SS no existe en DB tesorería,NaN,Aplica conciliación,2000,CD,NaN,NaN
1326,1335,12695968497,2000,TIENDA FALABELLA .COM,2022-06-28 21:44:12,1,8579,2022-06-30 00:00:00,2022-06-30,1,47708,F,HDREF,2022-06-28 21:44:21,RESERVADO,2022-07-01 00:28:35,ANULADO X NCRD,ANULADO X NCRD,NaN,EXPSRXCO,7702997567667,8820814,1.0,139900.0,139900.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,144029168791,370,263469,NaN,No tiene NC,Otro canal de venta,NaN,NaN,NaN,OVERDUE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [52]:
df_res['f12_desc_subestado'].unique()

array(['ENTREGA POR PDA', 'ENTREGA ADMINISTRATIVO',
       'RECHAZO ADM. CALL CENTER', 'ANULADO X FALTA', 'ENTREGA EN SRX',
       'ANULADO X USER', 'ANULADO X NCRD', 'EN LINEA PRV. CON FACTURA',
       nan, 'ENTREGA EN TIENDA', 'CON RECHAZO POS', 'EN RUTA AL CLIENTE',
       'EN RUTA CLIENTE DIRECTO/PROV', 'RECHAZADA X FALTA STOCK',
       'RECEPCION EN TRANSITO', 'BOLETEADO/FACTURADO', 'TOTAL ENTREGA',
       'ENTREGA PROVEEDOR', 'CONFIRMADO', 'RETIRADO DESDE PROVEEDOR',
       'EN TRANSITO', 'MT PROVEEDOR', 'MT', 'RESERVADO'], dtype=object)

In [46]:
mp.columns = mp.columns.str.strip()

array(['2022-11-03 00:00:00', '2022-11-02 00:00:00',
       '2022-11-01 00:00:00', '2022-11-04 00:00:00',
       '2022-11-05 00:00:00', '2022-11-07 00:00:00',
       '2022-11-06 00:00:00', '2022-06-24 00:00:00',
       '2022-06-25 00:00:00', '2022-06-27 00:00:00',
       '2022-06-28 00:00:00', '2022-06-18 00:00:00',
       '2022-06-22 00:00:00', '2022-06-29 00:00:00',
       '2022-06-30 00:00:00', '2022-06-14 00:00:00',
       '2022-06-01 00:00:00', '2022-06-02 00:00:00',
       '2022-06-03 00:00:00', '2022-06-04 00:00:00',
       '2022-06-06 00:00:00', '2022-06-07 00:00:00',
       '2022-06-08 00:00:00', '2022-06-09 00:00:00',
       '2022-06-11 00:00:00', '2022-06-10 00:00:00',
       '2022-06-23 00:00:00', '2022-06-16 00:00:00',
       '2022-06-21 00:00:00', '2022-06-15 00:00:00',
       '2022-06-17 00:00:00', '2022-06-13 00:00:00',
       '2022-06-26 00:00:00', '2022-08-05 00:00:00',
       '2022-06-19 00:00:00', '2022-08-16 00:00:00',
       '2022-06-05 00:00:00', '2022-05-28 00:0

In [47]:
mp[['FECHA DE PROCESO' , 'LOCAL', 'TERMINAL', 'SECUENCIA', 'MONTO']]

,FECHA DE PROCESO,LOCAL,TERMINAL,SECUENCIA,MONTO
0,2022-11-03 00:00:00,3000,3001,15109,-10490
1,2022-11-02 00:00:00,131,96,7709,-219988
2,2022-11-02 00:00:00,131,96,7709,-230002
3,2022-11-02 00:00:00,131,96,7709,-50000
4,2022-11-01 00:00:00,45,95,6095,-81000
...,...,...,...,...,...
624216,08-JAN-22,85,31,1361,-89990
624217,08-JAN-22,138,96,2182,-87920
624218,03-JAN-22,18,95,3452,-249900
624219,02-JAN-22,138,96,1071,-149994


In [ ]:
mp.columns = mp.columns.str.strip()

In [ ]:
mp.columns

Index(['Unnamed: 0', 'MEDIO DE PAGO', 'TARJETA', 'FECHA DE PROCESO',
       'FECHA DE VENTA', 'LOCAL', 'TERMINAL', 'SECUENCIA', 'MONTO',
       'NUM TARJETA', 'ESTADO', 'GLOSA DE MARCA', 'COD.AUTORIZACION',
       'TIPO TRANSACCION', 'DESC.DE TRANSACCION', 'COD.CONTRACARGO',
       'DESC.  CONTRACARGO', 'FECHA DE GASTO', 'IDENT REGISTRO', 'CORRELATIVO',
       'NUMERO DE CUOTA', 'CANTIDAD DE CUOTA', 'NUMERO DE DOCUMENTO', 'RUT',
       'HORA VENTA', 'FECHA PAGO(LIQ)', 'N° TARJETA REF.',
       'COD. AUTORIZA REF.', 'IDENTIFICADOR', 'FECHA VENTA (LIQ)',
       'LOCAL (LIQ)', 'COMERCIO (LIQ)', 'MICROFILM (LIQ)', 'N° TARJETA (LIQ)',
       'COD.AUTORIZACIÓN (LIQ)', 'USUARIO (LIQ)', 'FECHA VENTA (SRX)',
       'FECHA PROCESO (SRX)', 'LOCAL (SRX)', 'TIPO TRANSACC. (SRX)',
       'NCR.ORIGINAL', 'ADMIN PAREADA', 'TIPO TARJ.', 'COD TARJ.',
       'COD. VENDEDOR', 'NOMBRE VENDEDOR',
       'GUIDNUMERO SAPESTADO SAPMENSAJE SAP', 'Unnamed: 46', 'Unnamed: 47',
       'Unnamed: 48'],
      dtype='

In [ ]:
mp[['LOCAL', 'TERMINAL', 'SECUENCIA']]

,LOCAL,TERMINAL,SECUENCIA
0,3000,3001,15109
1,131,96,7709
2,131,96,7709
3,131,96,7709
4,45,95,6095
...,...,...,...
624216,85,31,1361
624217,138,96,2182
624218,18,95,3452
624219,138,96,1071


In [ ]:
df_res.loc[df_res['Novedad tesoreria'].notna()].shape

(13829, 132)

In [ ]:
for i in ['LOCAL', 'TERMINAL', 'SECUENCIA']:
    mp[i] = pd.to_numeric(mp[i])

In [ ]:
for i in ['f12_loc_nc', 'f12_nterminal', 'f12_nsecuencia']:
    df_res[i] = pd.to_numeric(df_res[i])

In [ ]:
mp

,Unnamed: 0,MEDIO DE PAGO,TARJETA,FECHA DE PROCESO,FECHA DE VENTA,LOCAL,TERMINAL,SECUENCIA,MONTO,NUM TARJETA,ESTADO,GLOSA DE MARCA,COD.AUTORIZACION,TIPO TRANSACCION,DESC.DE TRANSACCION,COD.CONTRACARGO,DESC. CONTRACARGO,FECHA DE GASTO,IDENT REGISTRO,CORRELATIVO,NUMERO DE CUOTA,CANTIDAD DE CUOTA,NUMERO DE DOCUMENTO,RUT,HORA VENTA,FECHA PAGO(LIQ),N° TARJETA REF.,COD. AUTORIZA REF.,IDENTIFICADOR,FECHA VENTA (LIQ),LOCAL (LIQ),COMERCIO (LIQ),MICROFILM (LIQ),N° TARJETA (LIQ),COD.AUTORIZACIÓN (LIQ),USUARIO (LIQ),FECHA VENTA (SRX),FECHA PROCESO (SRX),LOCAL (SRX),TIPO TRANSACC. (SRX),NCR.ORIGINAL,ADMIN PAREADA,TIPO TARJ.,COD TARJ.,COD. VENDEDOR,NOMBRE VENDEDOR,GUIDNUMERO SAPESTADO SAPMENSAJE SAP,Unnamed: 46,Unnamed: 47,Unnamed: 48
0,0,REC. LOGINSA,REC. LOGINSA,2022-11-03 00:00:00,2022-11-03 00:00:00,3000,3001,15109,-10490,1563088797,ACH,VTA 1-133635967565 1563088797,9272299792,91,NC ELECTRONICA,,No Defenido,1900-01-01,491196579,1,1,1,0,0,831,01-JAN-00,0,0,,01-JAN-00,0,0,0,0,0,0,01-JAN-00,01-JAN-00,0,0,8773,NO DEFINIDO,0,0,47708,FRANCISCO ANDRES MARTINEZ SANCHEZ,E860B62A3C802BA8E050010A59216B1E,1.602879e+09,Exito,El documento se ha contabilizado correctamente...
1,1,EFECTIVO,EFECTIVO,2022-11-02 00:00:00,2022-11-02 00:00:00,131,96,7709,-219988,0,ACH,1-133953036962,0,91,NC ELECTRONICA,,No Defenido,1900-01-01,491151042,1,0,0,0,0,2032,01-JAN-00,0,0,,01-JAN-00,0,0,0,0,0,0,01-JAN-00,01-JAN-00,0,0,0,NO DEFINIDO,NaN,0,151564,IRIS MARGARITA CUELLO PEREZ,E867C8EFC72DD54AE050010A59216BE2,1.000000e+00,Error,"Valor de Nombre, es obligatorio para escenario..."
2,2,TARJETA EXTERNA,O-OTROS,2022-11-02 00:00:00,2022-11-02 00:00:00,131,96,7709,-230002,9999,ACH,1-133953036962,19342834,91,NC ELECTRONICA,,No Defenido,1900-01-01,491151042,2,0,0,0,1018426446,2032,01-JAN-00,0,0,,01-JAN-00,0,0,0,0,0,0,01-JAN-00,01-JAN-00,0,0,3100,NO DEFINIDO,DB,93,151564,IRIS MARGARITA CUELLO PEREZ,E867C8EFC72DD54AE050010A59216BE2,1.000000e+00,Error,"Valor de Nombre, es obligatorio para escenario..."
3,3,TARJETA EXTERNA,O-OTROS,2022-11-02 00:00:00,2022-11-02 00:00:00,131,96,7709,-50000,9999,ACH,1-133953036962,19376727,91,NC ELECTRONICA,,No Defenido,1900-01-01,491151042,3,0,0,0,1018426446,2032,01-JAN-00,0,0,,01-JAN-00,0,0,0,0,0,0,01-JAN-00,01-JAN-00,0,0,3100,NO DEFINIDO,DB,93,151564,IRIS MARGARITA CUELLO PEREZ,E867C8EFC72DD54AE050010A59216BE2,1.000000e+00,Error,"Valor de Nombre, es obligatorio para escenario..."
4,4,TARJETA EXTERNA,O-OTROS,2022-11-01 00:00:00,2022-11-01 00:00:00,45,95,6095,-81000,9999,ACH,VTA 1-134299175853 0,17292578,91,NC ELECTRONICA,,No Defenido,1900-01-01,491125133,1,0,0,0,43812593,1428,01-JAN-00,0,0,,01-JAN-00,0,0,0,0,0,0,01-JAN-00,01-JAN-00,0,0,89731,NO DEFINIDO,DB,93,176796,PATRICIO ALEJANDRO MORIN MIRANDA,E8B9FA8E1A2147C8E050010A59212933,1.602999e+09,Exito,El documento se ha contabilizado correctamente...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
624216,624216,EFECTIVO,EFECTIVO,08-JAN-22,08-JAN-22,85,31,1361,-89990,0,Venta,,0,30,N/C (DEVOLUCION),,No Defenido,1900-01-01,478537965,1,0,0,0,0,1544,01-JAN-00,0,0,,01-JAN-00,0,0,0,0,0,0,01-JAN-00,01-JAN-00,0,0,0,NO DEFINIDO,NaN,0,926094,YARED CAROLINA MALDONADO CARPIO,NaN,NaN,NaN,NaN
624217,624217,EFECTIVO,EFECTIVO,08-JAN-22,08-JAN-22,138,96,2182,-87920,0,Venta,,0,30,N/C (DEVOLUCION),,No Defenido,1900-01-01,478544866,1,0,0,0,0,1752,01-JAN-00,0,0,,01-JAN-00,0,0,0,0,0,0,01-JAN-00,01-JAN-00,0,0,0,NO DEFINIDO,NaN,0,186899,SIN INFORMACION,NaN,NaN,NaN,NaN
624218,624218,EFECTIVO,EFECTIVO,03-JAN-22,03-JAN-22,18,95,3452,-249900,0,Venta,,0,30,N/C (DEVOLUCION),,No Defenido,1900-01-01,478302766,1,0,0,0,0,1525,01-JAN-00,0,0,,01-JAN-00,0,0,0,0,0,0,01-JAN-00,01-JAN-00,0,0,0,NO DEFINIDO,NaN,0,133086,EDINSON FABIAN FIGUEROA .,NaN,NaN,NaN,NaN
624219,624219,EFECTIVO,EFECTIVO,02-JAN-22,02-JAN-22,138,96,1071,-149994,0,Venta,,0,30,N/C (DEVOLUCION),,No Defenido,1900-01-01,4782

In [ ]:
mp.drop_duplicates(['LOCAL','MEDIO DE PAGO' , 'TERMINAL', 'SECUENCIA','MONTO', 'NUM TARJETA'], inplace = True)

In [ ]:
u = df_res.merge(mp, how='left', left_on= ['f12_loc_nc', 'f12_nterminal', 'f12_nsecuencia'], right_on=['LOCAL', 'TERMINAL', 'SECUENCIA'])

In [ ]:
u['Novedad tesoreria'].unique()

array([nan, 'No se encontro en base de tesoreria',
       'No aplica NC pero registra pago tesoreria', 'No Coindicide valor',
       'Canal de venta linio', 'Solución en tienda'], dtype=object)

In [ ]:
df_res.loc[(df_res['f12_loc_nc'] == 50) & (df_res['f12_nterminal'] == 3)]['gco_ind_pos'].unique()

array(['POS de SAC'], dtype=object)

In [ ]:
u.loc[(u['f12_gco_inc_nc'] == 'Tiene NC') & (u['gco_comment'] == 'Aplica conciliación') & (u['gco_ind_btes'] == 'La NC|SS existe en DB tesorería') & ((u['f12_gco_ind_nc_inst'].notna()) | (u['f12_ind_gift_card'].notna()) | (u['ind_pce'].notna())) & (u['MEDIO DE PAGO'] == 'TARJETA EXTERNA') ]

,index,f12_nfolio,f12_loc_id,f12_loc_name,f12_dcreacion,f12_caja_venta,f12_secuencia_venta,f12_dreparto,f12_dpactada,f12_ctipo,f12_cvendedor,f12_bretira_dsp,f12_origin,f12_fpestado,f12_pestado,f12_fuestado,f12_desc_estado,f12_desc_subestado,f12_desc_mt,f12_username,f12_prd_upc,f12_prod_sku_id,f12_qproducto,f12_mprecio,f12_tprecio,f12_mdescuento,f12_dproceso,f12_loc_nc,f12_nterminal,f12_nsecuencia,f12_monto_nc,f12_cautoriza,f12_trn_tech_key,f12_origin_nc,f12_nsuborden,f12_gco_inc_nc,f12_gco_ind_canal,f12_gco_ind_nc_inst,f12_ind_gift_card,gco_ind_fpactada,ss_ss,ss_n1,ss_n2,ss_n3,ss_tipo,ss_fcr,ss_estado,ss_subestado,ss_fecha_creacion,ss_fecha_solucion,ss_fecha_cierre,ss_tienda_origen,ss_orden_compra,ss_suborden,ss_num_f12,ss_area_problema,ss_sucursal_creador,ss_descripcion,ss_solucion,tesoreria_ntc_cod aut nc,tesoreria_ntc_ss,tesoreria_ntc_valor de devolucion,tesoreria_ntc_tipo de devolucion,estado_tesoreria,tesoreria_sieb_ss,tesoreria_sieb_valor de devolucion,tesoreria_sieb_tipo de devolucion,tesoreria_sieb_estado,ro_do_inicial,ro_upc,ro_fecha_creacion_ro,ro_ro,ro_estado_ro,ro_cantidad_producto,en_centrada,en_fcreareg,en_creferen,en_darticul,en_qcantida,mc_tip0_trabajo,mc_entrada,f11_nro_f11,f11_fecha_creacion,f11_upc,f11_propietario,f11_cod_local,f11_nombre_local,f11_grupo,f11_estado,f11_servicio,f11_cant_prod,f11_total_costo,f11_observacion,f11_folio_f12,f3_nro_devolucion,f3_fecha_reserva,f3_fecha_envio,f3_fecha_anulacion,f3_fecha_confirmacion,f3_tipo_producto,f3_upc,f3_sku,f3_descripcion,f3_local,f3_descripcion5,f3_estado,f3_descripcion6,f3_cantidad,f3_cant*costoprmd,f3_folio_f12,quiebres_f12,quiebres_sku,quiebres_cantidad_cancelada,quiebres_cost_unit_prod,quiebres_tipo_quiebre,quiebres_codigo_barras,q_man_fecha del quiebre,q_man_f12,q_man_sku,q_man_unidades,gco_ind_pos,ind_pce,gco_ind_entregas,gco_ind_ss,gco_ind_ss_n3,gco_ind_btes,gco_ind_registra_pago,gco_comment,LOC_ID,LOCAL_AGG,comentaroio_tes,Novedad tesoreria,Unnamed: 0,MEDIO DE PAGO,TARJETA,FECHA DE PROCESO,FECHA DE VENTA,LOCAL,TERMINAL,SECUENCIA,MONTO,NUM TARJETA,ESTADO,GLOSA DE MARCA,COD.AUTORIZACION,TIPO TRANSACCION,DESC.DE TRANSACCION,COD.CONTRACARGO,DESC. CONTRACARGO,FECHA DE GASTO,IDENT REGISTRO,CORRELATIVO,NUMERO DE CUOTA,CANTIDAD DE CUOTA,NUMERO DE DOCUMENTO,RUT,HORA VENTA,FECHA PAGO(LIQ),N° TARJETA REF.,COD. AUTORIZA REF.,IDENTIFICADOR,FECHA VENTA (LIQ),LOCAL (LIQ),COMERCIO (LIQ),MICROFILM (LIQ),N° TARJETA (LIQ),COD.AUTORIZACIÓN (LIQ),USUARIO (LIQ),FECHA VENTA (SRX),FECHA PROCESO (SRX),LOCAL (SRX),TIPO TRANSACC. (SRX),NCR.ORIGINAL,ADMIN PAREADA,TIPO TARJ.,COD TARJ.,COD. VENDEDOR,NOMBRE VENDEDOR,GUIDNUMERO SAPESTADO SAPMENSAJE SAP,Unnamed: 46,Unnamed: 47,Unnamed: 48
193312,193239,12690530842,2000,TIENDA FALABELLA .COM,2022-05-20 08:57:54,3461,81943,2022-05-24 00:00:00,2022-05-21,3,47708,F,HDREF,2022-05-20 08:57:57,RESERVADO,2022-05-26 12:25:01,ANULADO X NCRD,ANULADO X NCRD,NaN,EXPSRXCO,4064053694278,25620322,1.0,269990.0,269990.0,0.0,2022-05-26 04:00:00 UTC,13.0,95.0,2254.0,-269990,1078237442,233015426,BOLETA,144028251734,Tiene NC,Otro canal de venta,NaN,NaN,OVERDUE,1-126033320594,Boletas y Cobros,Devolución,DEV X NC EN TIENDA,Pedido,N,Cerrado,Finalizada,2022-05-27 00:00:00,2022-06-02 00:00:00,2022-06-02 00:00:00,Internet,1543960411,144028251734,12690530842,Boletas y Cobros,SAC Tienda Plaza Central Colombia,.cliente se acerca para proceso de normalizaci...,EN PROCESO ENVIADA A RED,1078237442,1-126033320594,269990.0,PARCIAL,PROCESADA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POS de SAC,y,NaN,Tiene SS,NaN,La NC|SS existe en DB tesorería,Pago procesado,Aplica conciliación,2000,CD,Coindicencia exacta,Solución en tienda,364093.0,TARJETA EXTERNA,O-OTROS,2022-05-26 00:00:00,2022-05-26 00:00:00,13.0,95.0,2254.0,-269990.0,9999.0,ACH,VTA 1-126033320594 1543960411,51452111,30.0,N/C (DEVOLUCIO

In [ ]:
u.loc[(u['Novedad tesoreria'] == 'Solución en tienda') & (u['MEDIO DE PAGO'] == 'TARJETA EXTERNA')]

,index,f12_nfolio,f12_loc_id,f12_loc_name,f12_dcreacion,f12_caja_venta,f12_secuencia_venta,f12_dreparto,f12_dpactada,f12_ctipo,f12_cvendedor,f12_bretira_dsp,f12_origin,f12_fpestado,f12_pestado,f12_fuestado,f12_desc_estado,f12_desc_subestado,f12_desc_mt,f12_username,f12_prd_upc,f12_prod_sku_id,f12_qproducto,f12_mprecio,f12_tprecio,f12_mdescuento,f12_dproceso,f12_loc_nc,f12_nterminal,f12_nsecuencia,f12_monto_nc,f12_cautoriza,f12_trn_tech_key,f12_origin_nc,f12_nsuborden,f12_gco_inc_nc,f12_gco_ind_canal,f12_gco_ind_nc_inst,f12_ind_gift_card,gco_ind_fpactada,ss_ss,ss_n1,ss_n2,ss_n3,ss_tipo,ss_fcr,ss_estado,ss_subestado,ss_fecha_creacion,ss_fecha_solucion,ss_fecha_cierre,ss_tienda_origen,ss_orden_compra,ss_suborden,ss_num_f12,ss_area_problema,ss_sucursal_creador,ss_descripcion,ss_solucion,tesoreria_ntc_cod aut nc,tesoreria_ntc_ss,tesoreria_ntc_valor de devolucion,tesoreria_ntc_tipo de devolucion,estado_tesoreria,tesoreria_sieb_ss,tesoreria_sieb_valor de devolucion,tesoreria_sieb_tipo de devolucion,tesoreria_sieb_estado,ro_do_inicial,ro_upc,ro_fecha_creacion_ro,ro_ro,ro_estado_ro,ro_cantidad_producto,en_centrada,en_fcreareg,en_creferen,en_darticul,en_qcantida,mc_tip0_trabajo,mc_entrada,f11_nro_f11,f11_fecha_creacion,f11_upc,f11_propietario,f11_cod_local,f11_nombre_local,f11_grupo,f11_estado,f11_servicio,f11_cant_prod,f11_total_costo,f11_observacion,f11_folio_f12,f3_nro_devolucion,f3_fecha_reserva,f3_fecha_envio,f3_fecha_anulacion,f3_fecha_confirmacion,f3_tipo_producto,f3_upc,f3_sku,f3_descripcion,f3_local,f3_descripcion5,f3_estado,f3_descripcion6,f3_cantidad,f3_cant*costoprmd,f3_folio_f12,quiebres_f12,quiebres_sku,quiebres_cantidad_cancelada,quiebres_cost_unit_prod,quiebres_tipo_quiebre,quiebres_codigo_barras,q_man_fecha del quiebre,q_man_f12,q_man_sku,q_man_unidades,gco_ind_pos,ind_pce,gco_ind_entregas,gco_ind_ss,gco_ind_ss_n3,gco_ind_btes,gco_ind_registra_pago,gco_comment,LOC_ID,LOCAL_AGG,comentaroio_tes,Novedad tesoreria,Unnamed: 0,MEDIO DE PAGO,TARJETA,FECHA DE PROCESO,FECHA DE VENTA,LOCAL,TERMINAL,SECUENCIA,MONTO,NUM TARJETA,ESTADO,GLOSA DE MARCA,COD.AUTORIZACION,TIPO TRANSACCION,DESC.DE TRANSACCION,COD.CONTRACARGO,DESC. CONTRACARGO,FECHA DE GASTO,IDENT REGISTRO,CORRELATIVO,NUMERO DE CUOTA,CANTIDAD DE CUOTA,NUMERO DE DOCUMENTO,RUT,HORA VENTA,FECHA PAGO(LIQ),N° TARJETA REF.,COD. AUTORIZA REF.,IDENTIFICADOR,FECHA VENTA (LIQ),LOCAL (LIQ),COMERCIO (LIQ),MICROFILM (LIQ),N° TARJETA (LIQ),COD.AUTORIZACIÓN (LIQ),USUARIO (LIQ),FECHA VENTA (SRX),FECHA PROCESO (SRX),LOCAL (SRX),TIPO TRANSACC. (SRX),NCR.ORIGINAL,ADMIN PAREADA,TIPO TARJ.,COD TARJ.,COD. VENDEDOR,NOMBRE VENDEDOR,GUIDNUMERO SAPESTADO SAPMENSAJE SAP,Unnamed: 46,Unnamed: 47,Unnamed: 48
193312,193239,12690530842,2000,TIENDA FALABELLA .COM,2022-05-20 08:57:54,3461,81943,2022-05-24 00:00:00,2022-05-21,3,47708,F,HDREF,2022-05-20 08:57:57,RESERVADO,2022-05-26 12:25:01,ANULADO X NCRD,ANULADO X NCRD,NaN,EXPSRXCO,4064053694278,25620322,1.0,269990.0,269990.0,0.0,2022-05-26 04:00:00 UTC,13.0,95.0,2254.0,-269990,1078237442,233015426,BOLETA,144028251734,Tiene NC,Otro canal de venta,NaN,NaN,OVERDUE,1-126033320594,Boletas y Cobros,Devolución,DEV X NC EN TIENDA,Pedido,N,Cerrado,Finalizada,2022-05-27 00:00:00,2022-06-02 00:00:00,2022-06-02 00:00:00,Internet,1543960411,144028251734,12690530842,Boletas y Cobros,SAC Tienda Plaza Central Colombia,.cliente se acerca para proceso de normalizaci...,EN PROCESO ENVIADA A RED,1078237442,1-126033320594,269990.0,PARCIAL,PROCESADA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POS de SAC,y,NaN,Tiene SS,NaN,La NC|SS existe en DB tesorería,Pago procesado,Aplica conciliación,2000,CD,Coindicencia exacta,Solución en tienda,364093.0,TARJETA EXTERNA,O-OTROS,2022-05-26 00:00:00,2022-05-26 00:00:00,13.0,95.0,2254.0,-269990.0,9999.0,ACH,VTA 1-126033320594 1543960411,51452111,30.0,N/C (DEVOLUCIO

In [ ]:
mp.loc[mp['MEDIO DE PAGO'].apply(lambda x : True if len(x) > 1 else False )]

,f12_cautoriza,MEDIO DE PAGO
5,1076846882,"[EFECTIVO, TARJETA GIFT]"
8,1076855051,"[EFECTIVO, VENTA CMR]"
12,1076872478,"[EFECTIVO, VENTA CMR]"
18,1076880206,"[EFECTIVO, VENTA CMR]"
25,1076899833,"[EFECTIVO, TARJETA GIFT]"
...,...,...
10336,1081655073,"[EFECTIVO, VENTA CMR]"
10340,1081668653,"[TARJETA EXTERNA, VENTA CMR]"
10344,1081687586,"[TARJETA EXTERNA, EFECTIVO]"
10354,1081703651,"[EFECTIVO, TARJETA EXTERNA]"


In [ ]:
u['MEDIO DE PAGO'].unique()

array([nan, 'REC. LOGINSA', 'EFECTIVO', 'TARJETA EXTERNA', 'QUICKPAY CMR',
       'GIFT CARD EMPRESA', 'QUICKPAY DEBITO BF', 'VENTA CMR',
       'TARJETA GIFT', 'FACT. POR COBRAR', 'TDE'], dtype=object)

In [ ]:
u['MEDIO DE PAGO'].unique()

array([nan, 'REC. LOGINSA', 'EFECTIVO', 'TARJETA EXTERNA', 'QUICKPAY CMR',
       'GIFT CARD EMPRESA', 'QUICKPAY DEBITO BF', 'VENTA CMR',
       'TARJETA GIFT', 'FACT. POR COBRAR', 'TDE'], dtype=object)

In [ ]:
['REC. LOGINSA', 'TARJETA EXTERNA', 'QUICKPAY CMR', 'QUICKPAY DEBITO BF', 'VENTA CMR']

['REC. LOGINSA',
 'TARJETA EXTERNA',
 'QUICKPAY CMR',
 'QUICKPAY DEBITO BF',
 'VENTA CMR']

In [ ]:
u.loc[u['MEDIO DE PAGO'] == 'REC. LOGINSA']

,index,f12_nfolio,f12_loc_id,f12_loc_name,f12_dcreacion,f12_caja_venta,f12_secuencia_venta,f12_dreparto,f12_dpactada,f12_ctipo,f12_cvendedor,f12_bretira_dsp,f12_origin,f12_fpestado,f12_pestado,f12_fuestado,f12_desc_estado,f12_desc_subestado,f12_desc_mt,f12_username,f12_prd_upc,f12_prod_sku_id,f12_qproducto,f12_mprecio,f12_tprecio,f12_mdescuento,f12_dproceso,f12_loc_nc,f12_nterminal,f12_nsecuencia,f12_monto_nc,f12_cautoriza,f12_trn_tech_key,f12_origin_nc,f12_nsuborden,f12_gco_inc_nc,f12_gco_ind_canal,f12_gco_ind_nc_inst,f12_ind_gift_card,gco_ind_fpactada,ss_ss,ss_n1,ss_n2,ss_n3,ss_tipo,ss_fcr,ss_estado,ss_subestado,ss_fecha_creacion,ss_fecha_solucion,ss_fecha_cierre,ss_tienda_origen,ss_orden_compra,ss_suborden,ss_num_f12,ss_area_problema,ss_sucursal_creador,ss_descripcion,ss_solucion,tesoreria_ntc_cod aut nc,tesoreria_ntc_ss,tesoreria_ntc_valor de devolucion,tesoreria_ntc_tipo de devolucion,estado_tesoreria,tesoreria_sieb_ss,tesoreria_sieb_valor de devolucion,tesoreria_sieb_tipo de devolucion,tesoreria_sieb_estado,ro_do_inicial,ro_upc,ro_fecha_creacion_ro,ro_ro,ro_estado_ro,ro_cantidad_producto,en_centrada,en_fcreareg,en_creferen,en_darticul,en_qcantida,mc_tip0_trabajo,mc_entrada,f11_nro_f11,f11_fecha_creacion,f11_upc,f11_propietario,f11_cod_local,f11_nombre_local,f11_grupo,f11_estado,f11_servicio,f11_cant_prod,f11_total_costo,f11_observacion,f11_folio_f12,f3_nro_devolucion,f3_fecha_reserva,f3_fecha_envio,f3_fecha_anulacion,f3_fecha_confirmacion,f3_tipo_producto,f3_upc,f3_sku,f3_descripcion,f3_local,f3_descripcion5,f3_estado,f3_descripcion6,f3_cantidad,f3_cant*costoprmd,f3_folio_f12,quiebres_f12,quiebres_sku,quiebres_cantidad_cancelada,quiebres_cost_unit_prod,quiebres_tipo_quiebre,quiebres_codigo_barras,q_man_fecha del quiebre,q_man_f12,q_man_sku,q_man_unidades,gco_ind_pos,ind_pce,gco_ind_entregas,gco_ind_ss,gco_ind_ss_n3,gco_ind_btes,gco_ind_registra_pago,gco_comment,LOC_ID,LOCAL_AGG,comentaroio_tes,Novedad tesoreria,Unnamed: 0,MEDIO DE PAGO,TARJETA,FECHA DE PROCESO,FECHA DE VENTA,LOCAL,TERMINAL,SECUENCIA,MONTO,NUM TARJETA,ESTADO,GLOSA DE MARCA,COD.AUTORIZACION,TIPO TRANSACCION,DESC.DE TRANSACCION,COD.CONTRACARGO,DESC. CONTRACARGO,FECHA DE GASTO,IDENT REGISTRO,CORRELATIVO,NUMERO DE CUOTA,CANTIDAD DE CUOTA,NUMERO DE DOCUMENTO,RUT,HORA VENTA,FECHA PAGO(LIQ),N° TARJETA REF.,COD. AUTORIZA REF.,IDENTIFICADOR,FECHA VENTA (LIQ),LOCAL (LIQ),COMERCIO (LIQ),MICROFILM (LIQ),N° TARJETA (LIQ),COD.AUTORIZACIÓN (LIQ),USUARIO (LIQ),FECHA VENTA (SRX),FECHA PROCESO (SRX),LOCAL (SRX),TIPO TRANSACC. (SRX),NCR.ORIGINAL,ADMIN PAREADA,TIPO TARJ.,COD TARJ.,COD. VENDEDOR,NOMBRE VENDEDOR,GUIDNUMERO SAPESTADO SAPMENSAJE SAP,Unnamed: 46,Unnamed: 47,Unnamed: 48
40,41,12688756728,2000,TIENDA FALABELLA .COM,2022-05-02 11:03:51,3463,45704,2022-05-06 00:00:00,2022-05-03,3,47708,F,HDREF,2022-05-02 11:03:57,RESERVADO,2022-05-11 15:23:26,TOTAL ENTREGA,ENTREGA ADMINISTRATIVO,RETORNADO A ORIGEN,YGALVRO,2998821976315,882197631,1.0,19990.0,19990.0,0.0,2022-05-14 04:00:00 UTC,3000.0,3001.0,73489.0,-81940,1077911855,232466961,BOLETA,144027999066,Tiene NC,Otro canal de venta,NaN,NaN,OVERDUE,1-124753083383,Despachos,Incumplimiento de fecha,INCUMPLIMIENTO SIN STOCK,Incidencia,N,Cerrado,Finalizada,2022-05-02 00:00:00,2022-05-16 00:00:00,2022-05-16 00:00:00,Internet,1541592711,144027999066,12688756728,Despachos,Siebel Administration,Actividades Proactivas: Solicitud de Servicio ...,RECHAZO A SAC PCE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MC,12688756728,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12688756728,882197631,1.0,19990.0,QUIEBRE PARCIAL,2998821976315,NaN,NaN,NaN,NaN,NaN,y,Tiene registro RO | MC | F3 | F11 | QUIEBRE,Tiene SS,NaN,La NC|SS no existe en DB tesorería,NaN,Aplica conciliación,2000,CD,NaN,NaN,25311.0,REC. LOGINSA,REC. LOGINSA,2022-05-14 00:00:00,2022-05-14 00:00:00,3000.0,3001.0,73489.0,-81940.0,1.541593e+09,Rev.Venta,,5121096979,91.0,NC ELECTRO

In [ ]:
df_res[(df_res['Novedad tesoreria'].notna()) & (df_res['f12_nfolio'] == '12698336187' )]

,index,f12_nfolio,f12_loc_id,f12_loc_name,f12_dcreacion,f12_caja_venta,f12_secuencia_venta,f12_dreparto,f12_dpactada,f12_ctipo,f12_cvendedor,f12_bretira_dsp,f12_origin,f12_fpestado,f12_pestado,f12_fuestado,f12_desc_estado,f12_desc_subestado,f12_desc_mt,f12_username,f12_prd_upc,f12_prod_sku_id,f12_qproducto,f12_mprecio,f12_tprecio,f12_mdescuento,f12_dproceso,f12_loc_nc,f12_nterminal,f12_nsecuencia,f12_monto_nc,f12_cautoriza,f12_trn_tech_key,f12_origin_nc,f12_nsuborden,f12_gco_inc_nc,f12_gco_ind_canal,f12_gco_ind_nc_inst,f12_ind_gift_card,gco_ind_fpactada,ss_ss,ss_n1,ss_n2,ss_n3,ss_tipo,ss_fcr,ss_estado,ss_subestado,ss_fecha_creacion,ss_fecha_solucion,ss_fecha_cierre,ss_tienda_origen,ss_orden_compra,ss_suborden,ss_num_f12,ss_area_problema,ss_sucursal_creador,ss_descripcion,ss_solucion,tesoreria_ntc_cod aut nc,tesoreria_ntc_ss,tesoreria_ntc_valor de devolucion,tesoreria_ntc_tipo de devolucion,estado_tesoreria,tesoreria_sieb_ss,tesoreria_sieb_valor de devolucion,tesoreria_sieb_tipo de devolucion,tesoreria_sieb_estado,ro_do_inicial,ro_upc,ro_fecha_creacion_ro,ro_ro,ro_estado_ro,ro_cantidad_producto,en_centrada,en_fcreareg,en_creferen,en_darticul,en_qcantida,mc_tip0_trabajo,mc_entrada,f11_nro_f11,f11_fecha_creacion,f11_upc,f11_propietario,f11_cod_local,f11_nombre_local,f11_grupo,f11_estado,f11_servicio,f11_cant_prod,f11_total_costo,f11_observacion,f11_folio_f12,f3_nro_devolucion,f3_fecha_reserva,f3_fecha_envio,f3_fecha_anulacion,f3_fecha_confirmacion,f3_tipo_producto,f3_upc,f3_sku,f3_descripcion,f3_local,f3_descripcion5,f3_estado,f3_descripcion6,f3_cantidad,f3_cant*costoprmd,f3_folio_f12,quiebres_f12,quiebres_sku,quiebres_cantidad_cancelada,quiebres_cost_unit_prod,quiebres_tipo_quiebre,quiebres_codigo_barras,q_man_fecha del quiebre,q_man_f12,q_man_sku,q_man_unidades,gco_ind_pos,ind_pce,gco_ind_entregas,gco_ind_ss,gco_ind_ss_n3,gco_ind_btes,gco_ind_registra_pago,gco_comment,LOC_ID,LOCAL_AGG,comentaroio_tes,Novedad tesoreria
37530,37788,12698336187,35,WTC CALI,2022-07-22 12:39:03,33,9751,2022-07-29 00:00:00,2022-07-29,2,161098,F,HDREF,2022-07-22 12:39:03,RESERVADO,2022-07-22 13:03:19,ANULADO X NCRD,ANULADO X NCRD,NaN,EXPSRXCO,8806092950030,30657373,2.0,2649900.0,5299800.0,-226800.0,2022-07-22 04:00:00 UTC,35.0,33.0,9752.0,-8333700,1079879467,236650720,BOLETA,NaN,Tiene NC,Otro canal de venta,NaN,NaN,OVERDUE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POS de whatsapp,NaN,NaN,No tiene SS,NaN,La NC|SS no existe en DB tesorería,NaN,Aplica conciliación,35,TIENDA,NaN,No se encontro en base de tesoreria
574792,578592,12698336187,35,WTC CALI,2022-07-22 12:39:03,33,9751,2022-07-29 00:00:00,2022-07-29,2,161098,F,HDREF,2022-07-22 12:39:03,RESERVADO,2022-07-22 13:03:19,ANULADO X NCRD,ANULADO X NCRD,NaN,EXPSRXCO,8801643787110,3833729,1.0,2899900.0,2899900.0,0.0,2022-07-22 04:00:00 UTC,35.0,33.0,9752.0,-8333700,1079879467,236650720,BOLETA,NaN,Tiene NC,Otro canal de venta,NaN,NaN,OVERDUE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POS de whatsapp,NaN,NaN,No tiene SS,NaN,La NC|SS no existe en DB tesorería,NaN,Aplica conciliación,35,TIENDA,NaN,No se encontro en base de tesoreria
1333897,1342618,12698336187,35,WTC CALI,2022-07-22 12:39:03,33,9751,2022-07-29 00:00:00,2022-07-29,2,161098,F,HDREF,2022-07-22 12:39:03,RESERVADO,2022-07-22 13:03:19,ANULADO X NCRD,ANULADO X NCRD,NaN,EXPSRXCO,2001069375007,1069375,1.0,134000.0,134000.0,0.0,2022-07-22 04:00:00 UTC,35.0,33.0,9752.0,-8333700,1079879467,236650720,BOLETA,NaN

In [ ]:
u.loc[u.duplicated(['f12_nfolio','f12_prod_sku_id', 'f12_qproducto'], keep=False) & (u['Novedad tesoreria'].notna())]

,index,f12_nfolio,f12_loc_id,f12_loc_name,f12_dcreacion,f12_caja_venta,f12_secuencia_venta,f12_dreparto,f12_dpactada,f12_ctipo,f12_cvendedor,f12_bretira_dsp,f12_origin,f12_fpestado,f12_pestado,f12_fuestado,f12_desc_estado,f12_desc_subestado,f12_desc_mt,f12_username,f12_prd_upc,f12_prod_sku_id,f12_qproducto,f12_mprecio,f12_tprecio,f12_mdescuento,f12_dproceso,f12_loc_nc,f12_nterminal,f12_nsecuencia,f12_monto_nc,f12_cautoriza,f12_trn_tech_key,f12_origin_nc,f12_nsuborden,f12_gco_inc_nc,f12_gco_ind_canal,f12_gco_ind_nc_inst,f12_ind_gift_card,gco_ind_fpactada,ss_ss,ss_n1,ss_n2,ss_n3,ss_tipo,ss_fcr,ss_estado,ss_subestado,ss_fecha_creacion,ss_fecha_solucion,ss_fecha_cierre,ss_tienda_origen,ss_orden_compra,ss_suborden,ss_num_f12,ss_area_problema,ss_sucursal_creador,ss_descripcion,ss_solucion,tesoreria_ntc_cod aut nc,tesoreria_ntc_ss,tesoreria_ntc_valor de devolucion,tesoreria_ntc_tipo de devolucion,estado_tesoreria,tesoreria_sieb_ss,tesoreria_sieb_valor de devolucion,tesoreria_sieb_tipo de devolucion,tesoreria_sieb_estado,ro_do_inicial,ro_upc,ro_fecha_creacion_ro,ro_ro,ro_estado_ro,ro_cantidad_producto,en_centrada,en_fcreareg,en_creferen,en_darticul,en_qcantida,mc_tip0_trabajo,mc_entrada,f11_nro_f11,f11_fecha_creacion,f11_upc,f11_propietario,f11_cod_local,f11_nombre_local,f11_grupo,f11_estado,f11_servicio,f11_cant_prod,f11_total_costo,f11_observacion,f11_folio_f12,f3_nro_devolucion,f3_fecha_reserva,f3_fecha_envio,f3_fecha_anulacion,f3_fecha_confirmacion,f3_tipo_producto,f3_upc,f3_sku,f3_descripcion,f3_local,f3_descripcion5,f3_estado,f3_descripcion6,f3_cantidad,f3_cant*costoprmd,f3_folio_f12,quiebres_f12,quiebres_sku,quiebres_cantidad_cancelada,quiebres_cost_unit_prod,quiebres_tipo_quiebre,quiebres_codigo_barras,q_man_fecha del quiebre,q_man_f12,q_man_sku,q_man_unidades,gco_ind_pos,ind_pce,gco_ind_entregas,gco_ind_ss,gco_ind_ss_n3,gco_ind_btes,gco_ind_registra_pago,gco_comment,LOC_ID,LOCAL_AGG,comentaroio_tes,Novedad tesoreria,Unnamed: 0,MEDIO DE PAGO,TARJETA,FECHA DE PROCESO,FECHA DE VENTA,LOCAL,TERMINAL,SECUENCIA,MONTO,NUM TARJETA,ESTADO,GLOSA DE MARCA,COD.AUTORIZACION,TIPO TRANSACCION,DESC.DE TRANSACCION,COD.CONTRACARGO,DESC. CONTRACARGO,FECHA DE GASTO,IDENT REGISTRO,CORRELATIVO,NUMERO DE CUOTA,CANTIDAD DE CUOTA,NUMERO DE DOCUMENTO,RUT,HORA VENTA,FECHA PAGO(LIQ),N° TARJETA REF.,COD. AUTORIZA REF.,IDENTIFICADOR,FECHA VENTA (LIQ),LOCAL (LIQ),COMERCIO (LIQ),MICROFILM (LIQ),N° TARJETA (LIQ),COD.AUTORIZACIÓN (LIQ),USUARIO (LIQ),FECHA VENTA (SRX),FECHA PROCESO (SRX),LOCAL (SRX),TIPO TRANSACC. (SRX),NCR.ORIGINAL,ADMIN PAREADA,TIPO TARJ.,COD TARJ.,COD. VENDEDOR,NOMBRE VENDEDOR,GUIDNUMERO SAPESTADO SAPMENSAJE SAP,Unnamed: 46,Unnamed: 47,Unnamed: 48
509,507,12693531150,101,CARACOLI,2022-06-17 07:46:47,35,8166,2022-06-28 00:00:00,2022-06-28,2,118125,F,HDREF,2022-06-17 07:46:47,RESERVADO,2022-06-17 17:43:00,ANULADO X NCRD,ANULADO X NCRD,NaN,EXPSRXCO,8806092804081,26173846,1.0,3609900.0,3609900.0,-738808.0,2022-06-17 04:00:00 UTC,101.0,35.0,8252.0,-4099890,1078780931,234326638,BOLETA,NaN,Tiene NC,Otro canal de venta,NaN,NaN,OVERDUE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POS piso de venta,NaN,NaN,No tiene SS,NaN,La NC|SS no existe en DB tesorería,NaN,Aplica conciliación,101,TIENDA,NaN,No se encontro en base de tesoreria,494303.0,VENTA CMR,VENTA CMR,2022-06-17 00:00:00,2022-06-17 00:00:00,101.0,35.0,8252.0,-2000000.0,1.150000e+02,Rev.Venta,,933297,30.0,N/C (DEVOLUCION),,No Defenido,1900-01-01,485448790.0,1.0,12.0,12.0,0.0,0.0,1742.0,01-JAN-00,115.0,933297,reversaco,01-JAN-00,0.0,0.0,0.0,0.0,0,0,2022-06-17 00:00:00,2022-06-17 00:00:00,101.0,1.0,0.0,NO DEFINIDO,0,0.0,118125.0,TAMARA ANDREA LETELIER NUÑEZ,NaN,NaN,NaN,NaN
510,507,12693531150,101,CARACOLI,2022-06-17 07:46:47,35

In [ ]:
u.loc[(u['f12_nfolio'] == '12698336187') & (u['Novedad tesoreria'].notna()),['f12_nfolio','f12_dproceso', 'f12_loc_nc', 'f12_nterminal', 'f12_nsecuencia', 'f12_monto_nc','FECHA DE PROCESO', 'MEDIO DE PAGO' ,'FECHA DE VENTA','LOCAL', 'TERMINAL', 'SECUENCIA', 'MONTO']].sort_values(by = 'MONTO')

,f12_nfolio,f12_dproceso,f12_loc_nc,f12_nterminal,f12_nsecuencia,f12_monto_nc,FECHA DE PROCESO,MEDIO DE PAGO,FECHA DE VENTA,LOCAL,TERMINAL,SECUENCIA,MONTO
1342967,12698336187,2022-07-22 04:00:00 UTC,35.0,33.0,9752.0,-8333700,2022-07-22 00:00:00,TARJETA GIFT,2022-07-22 00:00:00,35.0,33.0,9752.0,-2000000.0
1342965,12698336187,2022-07-22 04:00:00 UTC,35.0,33.0,9752.0,-8333700,2022-07-22 00:00:00,TARJETA GIFT,2022-07-22 00:00:00,35.0,33.0,9752.0,-2000000.0
37787,12698336187,2022-07-22 04:00:00 UTC,35.0,33.0,9752.0,-8333700,2022-07-22 00:00:00,TARJETA GIFT,2022-07-22 00:00:00,35.0,33.0,9752.0,-2000000.0
37788,12698336187,2022-07-22 04:00:00 UTC,35.0,33.0,9752.0,-8333700,2022-07-22 00:00:00,TARJETA GIFT,2022-07-22 00:00:00,35.0,33.0,9752.0,-2000000.0
37789,12698336187,2022-07-22 04:00:00 UTC,35.0,33.0,9752.0,-8333700,2022-07-22 00:00:00,TARJETA GIFT,2022-07-22 00:00:00,35.0,33.0,9752.0,-2000000.0
578734,12698336187,2022-07-22 04:00:00 UTC,35.0,33.0,9752.0,-8333700,2022-07-22 00:00:00,TARJETA GIFT,2022-07-22 00:00:00,35.0,33.0,9752.0,-2000000.0
578733,12698336187,2022-07-22 04:00:00 UTC,35.0,33.0,9752.0,-8333700,2022-07-22 00:00:00,TARJETA GIFT,2022-07-22 00:00:00,35.0,33.0,9752.0,-2000000.0
1342966,12698336187,2022-07-22 04:00:00 UTC,35.0,33.0,9752.0,-8333700,2022-07-22 00:00:00,TARJETA GIFT,2022-07-22 00:00:00,35.0,33.0,9752.0,-2000000.0
578732,12698336187,2022-07-22 04:00:00 UTC,35.0,33.0,9752.0,-8333700,2022-07-22 00:00:00,TARJETA GIFT,2022-07-22 00:00:00,35.0,33.0,9752.0,-2000000.0
1342964,12698336187,2022-07-22 04:00:00 UTC,35.0,33.0,9752.0,-8333700,2022-07-22 00:00:00,TARJETA GIFT,2022-07-22 00:00:00,35.0,33.0,9752.0,-1800000.0


In [ ]:
mp_gb = mp.groupby(['LOCAL','TERMINAL',	'SECUENCIA']).agg({'MONTO':'sum', 'MEDIO DE PAGO':'unique'}).reset_index()

In [ ]:
mp_gb['MONTO'] = abs(mp_gb['MONTO'])

In [ ]:
mp_gb.loc[mp_gb['MEDIO DE PAGO'].apply(lambda x : True if len(x) > 1 else  False )]

,LOCAL,TERMINAL,SECUENCIA,MONTO,MEDIO DE PAGO
202,6,5,3503,2436890,"[EFECTIVO, VENTA CMR]"
207,6,5,3935,709900,"[EFECTIVO, TARJETA EXTERNA]"
208,6,5,4049,7856895,"[EFECTIVO, VENTA CMR]"
209,6,5,4070,7856895,"[EFECTIVO, VENTA CMR]"
219,6,5,5259,1896390,"[EFECTIVO, VENTA CMR]"
...,...,...,...,...,...
417878,3000,3001,39770,14997,"[EFECTIVO, TARJETA EXTERNA]"
420965,3000,3001,42940,325763,"[EFECTIVO, TARJETA EXTERNA]"
426193,3000,3001,48216,761700,"[EFECTIVO, TARJETA EXTERNA]"
448831,3000,3001,71205,5740,"[TARJETA GIFT, QUICKPAY CMR]"


In [ ]:
mp[['LOCAL','TERMINAL',	'SECUENCIA']].drop_duplicates(['LOCAL','TERMINAL',	'SECUENCIA'])

,LOCAL,TERMINAL,SECUENCIA
0,3000,3001,15109
1,131,96,7709
4,45,95,6095
5,3000,3001,15352
6,138,96,2876
...,...,...,...
624214,36,14,2887
624215,85,17,7948
624216,85,31,1361
624218,18,95,3452


In [ ]:
mp.loc[(mp['LOCAL'] == 35) & (mp['TERMINAL']== 33) & (mp['SECUENCIA']== 9752)]['MONTO'].unique()

array([ -106900,  -200000, -1800000, -2000000], dtype=int64)

In [ ]:
mp['MONTO']

0         -10490
1        -219988
2        -230002
3         -50000
4         -81000
           ...  
624216    -89990
624217    -87920
624218   -249900
624219   -149994
624220   -159990
Name: MONTO, Length: 514092, dtype: int64

_______________________________________________

In [ ]:
df_res.head()

,index,f12_nfolio,f12_loc_id,f12_loc_name,f12_dcreacion,f12_caja_venta,f12_secuencia_venta,f12_dreparto,f12_dpactada,f12_ctipo,f12_cvendedor,f12_bretira_dsp,f12_origin,f12_fpestado,f12_pestado,f12_fuestado,f12_desc_estado,f12_desc_subestado,f12_desc_mt,f12_username,f12_prd_upc,f12_prod_sku_id,f12_qproducto,f12_mprecio,f12_tprecio,f12_mdescuento,f12_dproceso,f12_loc_nc,f12_nterminal,f12_nsecuencia,f12_monto_nc,f12_cautoriza,f12_trn_tech_key,f12_origin_nc,f12_nsuborden,f12_gco_inc_nc,f12_gco_ind_canal,f12_gco_ind_nc_inst,f12_gco_ind_gift_card,f12_gco_ind_te,gco_ind_fpactada,ss_ss,ss_n1,ss_n2,ss_n3,ss_tipo,ss_fcr,ss_estado,ss_subestado,ss_fecha_creacion,ss_fecha_solucion,ss_fecha_cierre,ss_tienda_origen,ss_orden_compra,ss_suborden,ss_num_f12,ss_area_problema,ss_sucursal_creador,ss_descripcion,ss_solucion,tesoreria_ntc_cod aut nc,tesoreria_ntc_ss,tesoreria_ntc_valor de devolucion,tesoreria_ntc_tipo de devolucion,estado_tesoreria,tesoreria_sieb_ss,tesoreria_sieb_valor de devolucion,tesoreria_sieb_tipo de devolucion,tesoreria_sieb_estado,ro_do_inicial,ro_upc,ro_fecha_creacion_ro,ro_ro,ro_estado_ro,ro_cantidad_producto,en_centrada,en_fcreareg,en_creferen,en_darticul,en_qcantida,mc_tip0_trabajo,mc_entrada,f11_nro_f11,f11_fecha_creacion,f11_upc,f11_propietario,f11_cod_local,f11_nombre_local,f11_grupo,f11_estado,f11_servicio,f11_cant_prod,f11_total_costo,f11_observacion,f11_folio_f12,f3_nro_devolucion,f3_fecha_reserva,f3_fecha_envio,f3_fecha_anulacion,f3_fecha_confirmacion,f3_tipo_producto,f3_upc,f3_sku,f3_descripcion,f3_local,f3_descripcion5,f3_estado,f3_descripcion6,f3_cantidad,f3_cant*costoprmd,f3_folio_f12,quiebres_f12,quiebres_sku,quiebres_cantidad_cancelada,quiebres_cost_unit_prod,quiebres_tipo_quiebre,quiebres_codigo_barras,q_man_fecha del quiebre,q_man_f12,q_man_sku,q_man_unidades,gco_ind_pos,ind_pce,gco_ind_entregas,gco_ind_ss,gco_ind_ss_n3,gco_ind_btes,gco_ind_registra_pago,gco_comment,LOC_ID,LOCAL_AGG,comentaroio_tes,Novedad tesoreria
0,0,12698020451,2000,TIENDA FALABELLA .COM,2022-07-19 14:29:28,3463.0,60455.0,2022-07-21 00:00:00,2022-07-21,3,47708.0,T,HDREF,2022-07-19 14:29:54,RESERVADO,2022-07-22 14:48:04,TOTAL ENTREGA,ENTREGA ADMINISTRATIVO,ENTREGA CLICK & COLLECT,EXPSRXCO,2024926456008.0,24926456.0,10.0,399900.0,399900.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,144029460126.0,No tiene NC,Otro canal de venta,NaN,NaN,NaN,OVERDUE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No tiene SS,NaN,La NC|SS no existe en DB tesorería,NaN,No aplíca conciliación,2000,CD,NaN,NaN
1,1,12695405053,2000,TIENDA FALABELLA .COM,2022-06-22 12:40:34,3462.0,21005.0,2022-06-24 00:00:00,2022-06-24,2,47708.0,T,HDREF,2022-06-22 12:40:37,RESERVADO,2022-06-25 14:23:48,TOTAL ENTREGA,ENTREGA ADMINISTRATIVO,ENTREGA CLICK & COLLECT,EXPSRXCO,3349668582860.0,20100981.0,10.0,3059920.0,3059920.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,144029076906.0,No tiene NC,Otro canal de venta,NaN,NaN,NaN,OVERDUE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No tiene SS,NaN,La NC|SS no existe en DB tesorería,NaN,No aplíca conciliación,2000,CD,NaN,NaN
2,2,12689452469,2000,TIENDA FALABELLA .COM,2022-05-07 10:29:17,NaN,NaN,2022-05-16 00:00:00,2022-05-10,2,NaN,F,HDREF,2022-05-07 10:29:21,RESERVADO,2022-05-07 11:48:50,ANULADO X USER,RECHAZO ADM. CALL CENTER,NaN,EXPSRXCO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,144028102427.0,No tiene NC,Otro canal de venta,NaN,NaN,NaN,OVERDUE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [ ]:
df_res['gco_ind_pos'].unique()

array([nan], dtype=object)

In [ ]:
df_res.loc[(df_res['f12_gco_ind_te'].notna()) & ()]

,index,f12_nfolio,f12_loc_id,f12_loc_name,f12_dcreacion,f12_caja_venta,f12_secuencia_venta,f12_dreparto,f12_dpactada,f12_ctipo,f12_cvendedor,f12_bretira_dsp,f12_origin,f12_fpestado,f12_pestado,f12_fuestado,f12_desc_estado,f12_desc_subestado,f12_desc_mt,f12_username,f12_prd_upc,f12_prod_sku_id,f12_qproducto,f12_mprecio,f12_tprecio,f12_mdescuento,f12_dproceso,f12_loc_nc,f12_nterminal,f12_nsecuencia,f12_monto_nc,f12_cautoriza,f12_trn_tech_key,f12_origin_nc,f12_nsuborden,f12_gco_inc_nc,f12_gco_ind_canal,f12_gco_ind_nc_inst,f12_gco_ind_gift_card,f12_gco_ind_te,gco_ind_fpactada,ss_ss,ss_n1,ss_n2,ss_n3,ss_tipo,ss_fcr,ss_estado,ss_subestado,ss_fecha_creacion,ss_fecha_solucion,ss_fecha_cierre,ss_tienda_origen,ss_orden_compra,ss_suborden,ss_num_f12,ss_area_problema,ss_sucursal_creador,ss_descripcion,ss_solucion,tesoreria_ntc_cod aut nc,tesoreria_ntc_ss,tesoreria_ntc_valor de devolucion,tesoreria_ntc_tipo de devolucion,estado_tesoreria,tesoreria_sieb_ss,tesoreria_sieb_valor de devolucion,tesoreria_sieb_tipo de devolucion,tesoreria_sieb_estado,ro_do_inicial,ro_upc,ro_fecha_creacion_ro,ro_ro,ro_estado_ro,ro_cantidad_producto,en_centrada,en_fcreareg,en_creferen,en_darticul,en_qcantida,mc_tip0_trabajo,mc_entrada,f11_nro_f11,f11_fecha_creacion,f11_upc,f11_propietario,f11_cod_local,f11_nombre_local,f11_grupo,f11_estado,f11_servicio,f11_cant_prod,f11_total_costo,f11_observacion,f11_folio_f12,f3_nro_devolucion,f3_fecha_reserva,f3_fecha_envio,f3_fecha_anulacion,f3_fecha_confirmacion,f3_tipo_producto,f3_upc,f3_sku,f3_descripcion,f3_local,f3_descripcion5,f3_estado,f3_descripcion6,f3_cantidad,f3_cant*costoprmd,f3_folio_f12,quiebres_f12,quiebres_sku,quiebres_cantidad_cancelada,quiebres_cost_unit_prod,quiebres_tipo_quiebre,quiebres_codigo_barras,q_man_fecha del quiebre,q_man_f12,q_man_sku,q_man_unidades,gco_ind_pos,ind_pce,gco_ind_entregas,gco_ind_ss,gco_ind_ss_n3,gco_ind_btes,gco_ind_registra_pago,gco_comment,LOC_ID,LOCAL_AGG,comentaroio_tes,Novedad tesoreria
346,351,12694424895,2000,TIENDA FALABELLA .COM,2022-06-17 19:00:55,3462.0,11345.0,2022-06-29 00:00:00,2022-06-24,2,47708.0,F,HDREF,2022-06-17 19:02:58,RESERVADO,2022-07-06 18:41:41,ANULADO X NCRD,ANULADO X NCRD,NaN,EXPSRXCO,194880483340.0,19829436.0,20.0,1441120.0,2882240.0,0.0,2022-07-06 00:00:00,123.0,18.0,3993.0,-302514.0,1079432020.0,235675693.0,BOLETA,144028942958.0,Tiene NC,Otro canal de venta,NaN,Solución en tienda GC,Y,OVERDUE,1-128262501398,Gestión sobre el producto,Devolución/Cambio de Compra,OMS - DERECHO RETRACTO-RETIRO,Pedido,N,Cerrado,Finalizada,2022-06-28 00:00:00,2022-07-11 00:00:00,2022-07-11 00:00:00,Internet,1550757930,144028942958,12694424895,Gestión sobre el producto,Siebel Administration,GUÍA A PUNTO SIN MOVIMIENTO.\nroxo - quedaron ...,Se finaliza ss RO en estado cancelado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tiene SS,NaN,La NC|SS no existe en DB tesorería,NaN,Aplica conciliación,2000,CD,NaN,NaN
464,471,12691633269,183,SANTA FE,2022-05-30 17:03:42,95.0,1497.0,2022-05-31 00:00:00,2022-05-31,2,121878.0,F,HDREF,2022-05-30 17:03:42,RESERVADO,2022-06-02 14:56:27,ANULADO X NCRD,ANULADO X NCRD,NaN,EXPSRXCO,2029697311006.0,29697311.0,10.0,7499000.0,7499000.0,4500000.0,2022-06-02 00:00:00,183.0,96.0,148.0,-749900.0,1078425511.0,233441083.0,BOLETA,NaN,Tiene NC,Otro canal de venta,NaN,NaN,Y,OVERDUE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No tiene SS,NaN,La NC|SS no existe en DB tesorería,NaN,Aplica conciliación,183,TIENDA,NaN,No se encontro en base de tesoreria
477,484,12686170007,2

In [ ]:
res = pd.read_csv('C:/Users/maperezr/OneDrive - Falabella/General/DATA/CONC/output/221030_resultado.csv', sep=';')

C:\Users\maperezr\AppData\Local\Temp\ipykernel_31696\581095538.py:1: DtypeWarning: Columns (36) have mixed types. Specify dtype option on import or set low_memory=False.
  res = pd.read_csv('C:/Users/maperezr/OneDrive - Falabella/General/DATA/CONC/output/221030_resultado.csv', sep=';')


In [ ]:
res.head()

,f12_nfolio,f12_loc_id,f12_loc_name,f12_dcreacion,f12_caja_venta,f12_secuencia_venta,f12_dreparto,f12_dpactada,f12_ctipo,f12_cvendedor,f12_bretira_dsp,f12_origin,f12_fpestado,f12_pestado,f12_fuestado,f12_desc_estado,f12_desc_subestado,f12_desc_mt,f12_username,f12_prd_upc,f12_prod_sku_id,f12_qproducto,f12_mprecio,f12_tprecio,f12_mdescuento,f12_dproceso,f12_loc_nc,f12_nterminal,f12_nsecuencia,f12_monto_nc,f12_cautoriza,f12_trn_tech_key,f12_origin_nc,f12_nsuborden,f12_gco_inc_nc,f12_gco_ind_canal,f12_gco_ind_nc_inst,f12_ind_gift_card,gco_ind_entregas,gco_ind_ss,gco_ind_ss_n3,gco_comment,gco_ind_btes,estado_tesoreria,gco_ind_registra_pago,ind_pce,LOCAL_AGG,comentaroio_tes
0,12697452765,2000,TIENDA FALABELLA .COM,2022-07-14 19:48:35,3462.0,47152.0,2022-07-19 00:00:00,2022-07-16,3,46604.0,F,HDREF,2022-07-14 19:48:45,RESERVADO,2022-07-15 13:31:24,TOTAL ENTREGA,ENTREGA POR PDA,SIN PROBLEMA ENTREGA EN LINEA,EXPSRXCO,6.281556e+11,23872769.0,1.0,144000.0,144000.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.440294e+11,No tiene NC,Otro canal de venta,NaN,NaN,NaN,No tiene SS,NaN,No aplíca conciliación,La NC|SS no existe en DB tesorería,NaN,NaN,NaN,CD,NaN
1,12697252464,2000,TIENDA FALABELLA .COM,2022-07-13 08:41:57,NaN,NaN,2022-07-15 00:00:00,2022-07-15,3,NaN,T,HDREF,2022-07-13 08:42:09,RESERVADO,2022-07-13 13:31:20,ANULADO X USER,ANULADO X USER,NaN,EXPSRXCO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.440294e+11,No tiene NC,Otro canal de venta,NaN,NaN,NaN,No tiene SS,NaN,No aplíca conciliación,La NC|SS no existe en DB tesorería,NaN,NaN,NaN,CD,NaN
2,12699106864,2000,TIENDA FALABELLA .COM,2022-07-30 19:42:42,NaN,NaN,2022-07-30 00:00:00,2022-07-30,2,NaN,T,HDREF,2022-07-30 19:42:46,RESERVADO,2022-07-30 21:46:40,ANULADO X USER,ANULADO X USER,NaN,EXPSRXCO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.440296e+11,No tiene NC,Otro canal de venta,NaN,NaN,NaN,No tiene SS,NaN,No aplíca conciliación,La NC|SS no existe en DB tesorería,NaN,NaN,NaN,CD,NaN
3,12691987751,2000,TIENDA FALABELLA .COM,2022-06-04 23:19:37,NaN,NaN,2022-06-09 00:00:00,2022-06-09,2,NaN,T,HDREF,2022-06-04 23:19:46,RESERVADO,2022-06-04 23:30:17,ANULADO X USER,RECHAZO ADM. CALL CENTER,NaN,EXPSRXCO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.440285e+11,No tiene NC,Otro canal de venta,NaN,NaN,NaN,No tiene SS,NaN,No aplíca conciliación,La NC|SS no existe en DB tesorería,NaN,NaN,NaN,CD,NaN
4,12686995259,3000,FALABELLA. COM ALTERNO,2022-04-09 08:08:43,20.0,3108.0,2022-04-10 00:00:00,2022-04-10,2,965588.0,F,HDREF,2022-04-09 08:08:43,RESERVADO,2022-04-11 16:12:30,TOTAL ENTREGA,ENTREGA POR PDA,SIN PROBLEMA ENTREGA EN LINEA,EXPSRXCO,2.000951e+12,951007.0,1.0,18000.0,18000.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No tiene NC,Otro canal de venta,NaN,NaN,NaN,No tiene SS,NaN,No aplíca conciliación,La NC|SS no existe en DB tesorería,NaN,NaN,NaN,CD,NaN


In [ ]:
res.loc[res['f12_cautoriza'] == 1070970699]

,f12_nfolio,f12_loc_id,f12_loc_name,f12_dcreacion,f12_caja_venta,f12_secuencia_venta,f12_dreparto,f12_dpactada,f12_ctipo,f12_cvendedor,f12_bretira_dsp,f12_origin,f12_fpestado,f12_pestado,f12_fuestado,f12_desc_estado,f12_desc_subestado,f12_desc_mt,f12_username,f12_prd_upc,f12_prod_sku_id,f12_qproducto,f12_mprecio,f12_tprecio,f12_mdescuento,f12_dproceso,f12_loc_nc,f12_nterminal,f12_nsecuencia,f12_monto_nc,f12_cautoriza,f12_trn_tech_key,f12_origin_nc,f12_nsuborden,f12_gco_inc_nc,f12_gco_ind_canal,f12_gco_ind_nc_inst,f12_ind_gift_card,gco_ind_entregas,gco_ind_ss,gco_ind_ss_n3,gco_comment,gco_ind_btes,estado_tesoreria,gco_ind_registra_pago,ind_pce,LOCAL_AGG,comentaroio_tes
